# データサイエンス100本ノック（構造化データ加工編） - Python

## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートとデータベース（PostgreSQL）からのデータ読み込みを行います
- pandas等、利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（"!pip install ライブラリ名"でインストールも可能）
- 処理は複数回に分けても構いません
- 名前、住所等はダミーデータであり、実在するものではありません

In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import math
import psycopg2
from sqlalchemy import create_engine
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler

pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    'user': os.environ['PG_USER'],
    'password': os.environ['PG_PASSWORD'],
}

# pd.read_sql用のコネクタ
conn = psycopg2.connect(**pgconfig)

df_customer = pd.read_sql(sql='select * from customer', con=conn)
df_category = pd.read_sql(sql='select * from category', con=conn)
df_product = pd.read_sql(sql='select * from product', con=conn)
df_receipt = pd.read_sql(sql='select * from receipt', con=conn)
df_store = pd.read_sql(sql='select * from store', con=conn)
df_geocode = pd.read_sql(sql='select * from geocode', con=conn)

# Ver1.1追加設定
> ・query, groupbyでのrename, aggの辞書渡しは使用しない

## Ver1.1備考
> ・修正のある問題は、前回の回答（ver1.0）に続いてVer1.1と記載\
> ・簡易な問題は飛ばす（セル削除）

## わかったこと
> ・queryは使わないほうが速い \
> ・JupyterLabは[ctrl]+[Z]でセルごと戻せる

# 演習問題

---
> P-001: レシート明細のデータフレーム（df_receipt）から全項目の先頭10件を表示し、どのようなデータを保有しているか目視で確認せよ。

---
> P-002: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示させよ。

---
> P-003: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示させよ。ただし、sales_ymdはsales_dateに項目名を変更しながら抽出すること。

---
> P-004: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"

---
> P-005: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上

---
> P-006: レシート明細データフレーム「df_receipt」から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

---
> P-007: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上2,000以下

---
> P-008: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 商品コード（product_cd）が"P071401019"以外

---
> P-009: 以下の処理において、出力結果を変えずにORをANDに書き換えよ。

`df_store.query('not(prefecture_cd == "13" | floor_area > 900)')`

---
> P-010: 店舗データフレーム（df_store）から、店舗コード（store_cd）が"S14"で始まるものだけ全項目抽出し、10件だけ表示せよ。

In [27]:
%%time
df_store.query(
    
    "store_cd.str.startswith('S14')",
    
    engine="python"
    
).head(10)

CPU times: user 2.52 ms, sys: 848 µs, total: 3.37 ms
Wall time: 3 ms


,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
4,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0
7,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
9,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
12,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0
16,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341.0
18,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
19,S14022,逗子店,14,神奈川県,神奈川県逗子市逗子一丁目,カナガワケンズシシズシイッチョウメ,046-123-4036,139.5789,35.29642,1838.0
20,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0


## ver1.1
> **queryよりも処理速度が速い**

In [28]:
%%time
df_store[
    
    # 店舗コードの先頭がS14
    df_store["store_cd"].str.startswith("S14")
    
].head(10)

CPU times: user 845 µs, sys: 283 µs, total: 1.13 ms
Wall time: 890 µs


,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
4,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0
7,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
9,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
12,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0
16,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341.0
18,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
19,S14022,逗子店,14,神奈川県,神奈川県逗子市逗子一丁目,カナガワケンズシシズシイッチョウメ,046-123-4036,139.5789,35.29642,1838.0
20,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0


---
> P-011: 顧客データフレーム（df_customer）から顧客ID（customer_id）の末尾が1のものだけ全項目抽出し、10件だけ表示せよ。

In [15]:
df_customer.query(
    
    'customer_id.str.endswith("1")',
    
    engine="python"
    
).head(10)

## Ver1.1

In [16]:
df_customer[
    
    # 顧客IDの末尾が１
    df_customer["customer_id"].str.endswith("1")
    
].head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
3,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
14,CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
31,CS028314000011,小菅 あおい,1,女性,1983-11-26,35,246-0038,神奈川県横浜市瀬谷区宮沢**********,S14028,20151123,1-20080426-5
56,CS039212000051,藤島 恵梨香,1,女性,1997-02-03,22,166-0001,東京都杉並区阿佐谷北**********,S13039,20171121,1-20100215-4
59,CS015412000111,松居 奈月,1,女性,1972-10-04,46,136-0071,東京都江東区亀戸**********,S13015,20150629,0-00000000-0
63,CS004702000041,野島 洋,0,男性,1943-08-24,75,176-0022,東京都練馬区向山**********,S13004,20170218,0-00000000-0
74,CS041515000001,栗田 千夏,1,女性,1967-01-02,52,206-0001,東京都多摩市和田**********,S13041,20160422,E-20100803-F
85,CS029313000221,北条 ひかり,1,女性,1987-06-19,31,279-0011,千葉県浦安市美浜**********,S12029,20180810,0-00000000-0
102,CS034312000071,望月 奈央,1,女性,1980-09-20,38,213-0026,神奈川県川崎市高津区久末**********,S14034,20160106,0-00000000-0


---
> P-012: 店舗データフレーム（df_store）から横浜市の店舗だけ全項目表示せよ。

In [29]:
df_store.query(
    
    'address.str.contains("横浜市")',
    
    engine="python"
    
)

## Ver1.1

In [25]:
%%time
df_store[
    
    # 住所が横浜市のみ
    df_store["address"].str.contains("横浜市")
    
]

CPU times: user 659 µs, sys: 220 µs, total: 879 µs
Wall time: 738 µs


,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
7,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
9,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
12,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0
16,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341.0
18,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
20,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0
26,S14048,中川中央店,14,神奈川県,神奈川県横浜市都筑区中川中央二丁目,カナガワケンヨコハマシツヅキクナカガワチュウオウニチョウメ,045-123-4051,139.5758,35.54912,1657.0
40,S14042,新山下店,14,神奈川県,神奈川県横浜市中区新山下二丁目,カナガワケンヨコハマシナカクシンヤマシタニチョウメ,045-123-4047,139.6593,35.43894,1044.0


---
> P-013: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まるデータを全項目抽出し、10件だけ表示せよ。

In [30]:
df_customer.query(
    
    'status_cd.str.contains("^[A-Z].*", regex=True)',
    
    engine="python"
    
).head(10)

## Ver1.1

In [23]:
%%time
df_customer[
    
    # ステータスコードの先頭がA~Z
    df_customer["status_cd"].str.contains("^[A-Z]", regex=True)
    
].head(10)

CPU times: user 21 ms, sys: 142 µs, total: 21.1 ms
Wall time: 18.7 ms


,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
6,CS015414000103,奥野 陽子,1,女性,1977-08-09,41,136-0073,東京都江東区北砂**********,S13015,20150722,B-20100609-B
12,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
15,CS029415000023,梅田 里穂,1,女性,1976-01-17,43,279-0043,千葉県浦安市富士見**********,S12029,20150610,D-20100918-E
21,CS035415000029,寺沢 真希,9,不明,1977-09-27,41,158-0096,東京都世田谷区玉川台**********,S13035,20141220,F-20101029-F
32,CS031415000106,宇野 由美子,1,女性,1970-02-26,49,151-0053,東京都渋谷区代々木**********,S13031,20150201,F-20100511-E
33,CS029215000025,石倉 美帆,1,女性,1993-09-28,25,279-0022,千葉県浦安市今川**********,S12029,20150708,B-20100820-C
40,CS033605000005,猪股 雄太,0,男性,1955-12-05,63,246-0031,神奈川県横浜市瀬谷区瀬谷**********,S14033,20150425,F-20100917-E
44,CS033415000229,板垣 菜々美,1,女性,1977-11-07,41,246-0021,神奈川県横浜市瀬谷区二ツ橋町**********,S14033,20150712,F-20100326-E
53,CS008415000145,黒谷 麻緒,1,女性,1977-06-27,41,157-0067,東京都世田谷区喜多見**********,S13008,20150829,F-20100622-F


---
> P-014: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の末尾が数字の1〜9で終わるデータを全項目抽出し、10件だけ表示せよ。

In [21]:
df_customer.query(
    
    'status_cd.str.contains(".*[1-9]$", regex=True)',
    
    engine="python"
    
).head(10)

## Ver1.1

In [31]:
df_customer[
 
    # ステータスコードの末尾が1~9
    df_customer["status_cd"].str.contains("[1-9]$", regex=True)
    
].head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
4,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2
9,CS033513000180,安斎 遥,1,女性,1962-07-11,56,241-0823,神奈川県横浜市旭区善部町**********,S14033,20150728,6-20080506-5
12,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
14,CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
16,CS009315000023,皆川 文世,1,女性,1980-04-15,38,154-0012,東京都世田谷区駒沢**********,S13009,20150319,5-20080322-1
22,CS015315000033,福士 璃奈子,1,女性,1983-03-17,36,135-0043,東京都江東区塩浜**********,S13015,20141024,4-20080219-3
23,CS023513000066,神戸 そら,1,女性,1961-12-17,57,210-0005,神奈川県川崎市川崎区東田町**********,S14023,20150915,5-20100524-9
24,CS035513000134,市川 美帆,1,女性,1960-03-27,59,156-0053,東京都世田谷区桜**********,S13035,20150227,8-20100711-9
27,CS001515000263,高松 夏空,1,女性,1962-11-09,56,144-0051,東京都大田区西蒲田**********,S13001,20160812,1-20100804-1
28,CS040314000027,鶴田 きみまろ,9,不明,1986-03-26,33,226-0027,神奈川県横浜市緑区長津田**********,S14040,20150122,2-20080426-4


---
> P-015: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まり、末尾が数字の1〜9で終わるデータを全項目抽出し、10件だけ表示せよ。

In [32]:
df_customer.query(
    
    'status_cd.str.contains("^[A-F].*[1-9]$", regex=True)', engine="python"
    
).head(10)

## Ver1.1

In [33]:
df_customer[
    
    # ステータスコードの先頭A~Z、末尾1~9
    df_customer["status_cd"].str.contains("^[A-F].*[1-9]$", regex=True)
    
].head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
12,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
68,CS022513000105,島村 貴美子,1,女性,1962-03-12,57,249-0002,神奈川県逗子市山の根**********,S14022,20150320,A-20091115-7
71,CS001515000096,水野 陽子,9,不明,1960-11-29,58,144-0053,東京都大田区蒲田本町**********,S13001,20150614,A-20100724-7
122,CS013615000053,西脇 季衣,1,女性,1953-10-18,65,261-0026,千葉県千葉市美浜区幕張西**********,S12013,20150128,B-20100329-6
144,CS020412000161,小宮 薫,1,女性,1974-05-21,44,174-0042,東京都板橋区東坂下**********,S13020,20150822,B-20081021-3
178,CS001215000097,竹中 あさみ,1,女性,1990-07-25,28,146-0095,東京都大田区多摩川**********,S13001,20170315,A-20100211-2
252,CS035212000007,内村 恵梨香,1,女性,1990-12-04,28,152-0023,東京都目黒区八雲**********,S13035,20151013,B-20101018-6
259,CS002515000386,野田 コウ,1,女性,1963-05-30,55,185-0013,東京都国分寺市西恋ケ窪**********,S13002,20160410,C-20100127-8
293,CS001615000372,稲垣 寿々花,1,女性,1956-10-29,62,144-0035,東京都大田区南蒲田**********,S13001,20170403,A-20100104-1
297,CS032512000121,松井 知世,1,女性,1962-09-04,56,210-0011,神奈川県川崎市川崎区富士見**********,S13032,20150727,A-20100103-5


---
> P-016: 店舗データフレーム（df_store）から、電話番号（tel_no）が3桁-3桁-4桁のデータを全項目表示せよ。

In [34]:
df_store.query(
    
    'tel_no.str.contains("^[0-9]{3}-[0-9]{3}-[0-9]{4}$", regex=True)', engine="python"
    
)

## Ver1.1

In [35]:
df_store[
    
    # 電話番号が3-3-4
    df_store["tel_no"].str.contains("^[0-9]{3}-[0-9]{3}-[0-9]{4}$", regex=True)
    
]

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
0,S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.1180,35.63559,1698.0
1,S13002,国分寺店,13,東京都,東京都国分寺市本多二丁目,トウキョウトコクブンジシホンダニチョウメ,042-123-4008,139.4802,35.70566,1735.0
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
4,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0
7,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
9,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
11,S13052,森野店,13,東京都,東京都町田市森野三丁目,トウキョウトマチダシモリノサンチョウメ,042-123-4030,139.4383,35.55293,1087.0
12,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0
16,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341.0


---
> P-17: 顧客データフレーム（df_customer）を生年月日（birth_day）で高齢順にソートし、先頭10件を全項目表示せよ。

---
> P-18: 顧客データフレーム（df_customer）を生年月日（birth_day）で若い順にソートし、先頭10件を全項目表示せよ。

---
> P-19: レシート明細データフレーム（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭10件を抽出せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合は同一順位を付与するものとする。

In [38]:
%%time
_tmp_df = pd.concat(
    
            [
                df_receipt[['customer_id', 'amount']], 
                pd.DataFrame({"rank_amount":df_receipt["amount"].rank(ascending=False, method="min")})
                
            ],
    
            axis=1
    
        )
_tmp_df.sort_values("rank_amount", ascending=True).head(10)

CPU times: user 33.9 ms, sys: 18.5 ms, total: 52.4 ms
Wall time: 49.7 ms


,customer_id,amount,rank_amount
1202,CS011415000006,10925,1.0
62317,ZZ000000000000,6800,2.0
54095,CS028605000002,5780,3.0
4632,CS015515000034,5480,4.0
72747,ZZ000000000000,5480,4.0
10320,ZZ000000000000,5480,4.0
97294,CS021515000089,5440,7.0
28304,ZZ000000000000,5440,7.0
92246,CS009415000038,5280,9.0
68553,CS040415000200,5280,9.0


## Ver1.1

In [42]:
%time
# 顧客ID、売上金額をコピー
_df = df_receipt[["customer_id", "amount"]].copy()

# 売上金額のランクを計算
# ・同じ順位（順位飛ばしあり）
# ・順位は整数に直す
_df["rank"] = _df["amount"].rank(method="min", ascending=False).astype('int')

_df.sort_values("rank", ascending=True).head(10)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.63 µs


,customer_id,amount,rank
1202,CS011415000006,10925,1
62317,ZZ000000000000,6800,2
54095,CS028605000002,5780,3
4632,CS015515000034,5480,4
72747,ZZ000000000000,5480,4
10320,ZZ000000000000,5480,4
97294,CS021515000089,5440,7
28304,ZZ000000000000,5440,7
92246,CS009415000038,5280,9
68553,CS040415000200,5280,9


**（松尾コメント）**\
pd.concatの処理およびpd.DataFrameへの変換がない分、処理速度が速い

---
> P-020: レシート明細データフレーム（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭10件を抽出せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合でも別順位を付与すること。

In [43]:
_tmp_df = pd.concat(
    
            [
                df_receipt[['customer_id', 'amount']],
                
                pd.DataFrame(
                    {"rnk_amount":df_receipt["amount"].rank(ascending=False, method="first")}
                )
            ],
            axis=1

)
_tmp_df.sort_values("rnk_amount", ascending=True).head(10)

## Ver1.1

In [45]:
# 顧客ID,売上金額をコピー
_df = df_receipt[["customer_id", "amount"]].copy()

# 売上金額のランクを計算
# ・同値でも別順位
_df["rank"] = _df["amount"].rank(method="first", ascending=False).astype('int')

_df.sort_values("rank", ascending=True).head(10)

,customer_id,amount,rank
1202,CS011415000006,10925,1
62317,ZZ000000000000,6800,2
54095,CS028605000002,5780,3
4632,CS015515000034,5480,4
10320,ZZ000000000000,5480,5
72747,ZZ000000000000,5480,6
28304,ZZ000000000000,5440,7
97294,CS021515000089,5440,8
596,CS015515000083,5280,9
11275,CS017414000114,5280,10


---
> P-021: レシート明細データフレーム（df_receipt）に対し、件数をカウントせよ。

---
> P-022: レシート明細データフレーム（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

---
> P-023: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）と売上数量（quantity）を合計せよ。

In [57]:
%%time
df_receipt.groupby("store_cd").agg(
    
        {"amount":sum, "quantity":sum}

).rename(
    
    {"amount":"sum_amt", "quantity":"sum_qnt"}
    
).reset_index().head(10)

CPU times: user 24.5 ms, sys: 0 ns, total: 24.5 ms
Wall time: 21.7 ms


,store_cd,amount,quantity
0,S12007,638761,2099
1,S12013,787513,2425
2,S12014,725167,2358
3,S12029,794741,2555
4,S12030,684402,2403
5,S13001,811936,2347
6,S13002,727821,2340
7,S13003,764294,2197
8,S13004,779373,2390
9,S13005,629876,2004


## Ver1.1
> 比較のため10件表示

In [58]:
%%time
# 集約処理関数を定義
def def_grp(x):
    d = {}
    d["sum_amt"] = x["amount"].sum()
    d["sum_qnt"] = x["quantity"].sum()
    
    return pd.Series(d)
    
df_receipt.groupby("store_cd").apply(def_grp).reset_index().head(10)

CPU times: user 69 ms, sys: 1.09 ms, total: 70.1 ms
Wall time: 64.8 ms


,store_cd,sum_amt,sum_qnt
0,S12007,638761,2099
1,S12013,787513,2425
2,S12014,725167,2358
3,S12029,794741,2555
4,S12030,684402,2403
5,S13001,811936,2347
6,S13002,727821,2340
7,S13003,764294,2197
8,S13004,779373,2390
9,S13005,629876,2004


**（松尾コメント）**\
groupby使用時にapplyを使うことで\
・メリット　：**そもそも辞書渡しは廃止予定**、集約後の列名変更が楽\
・デメリット：処理時間増加

---
> P-024: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上日（sales_ymd）を求め、10件表示せよ。

In [59]:
%%time
df_receipt.groupby("customer_id").max("sales_ymd").reset_index().head(10)

CPU times: user 41.3 ms, sys: 10.3 ms, total: 51.7 ms
Wall time: 48.1 ms


,customer_id,sales_ymd,sales_epoch,receipt_no,receipt_sub_no,quantity,amount
0,CS001113000004,20190308,1552003200,112,2,1,1200
1,CS001114000005,20190731,1564531200,112,2,1,338
2,CS001115000010,20190405,1554422400,112,2,1,2200
3,CS001205000004,20190625,1561420800,1192,2,1,760
4,CS001205000006,20190224,1550966400,1192,2,1,1100
5,CS001211000025,20190322,1553212800,112,2,1,298
6,CS001212000027,20170127,1485475200,112,2,1,248
7,CS001212000031,20180906,1536192000,112,2,1,148
8,CS001212000046,20170811,1502409600,112,2,1,120
9,CS001212000070,20191018,1571356800,112,2,1,308


## Ver1.1

In [60]:
%%time
def def_grp(x):
    d={}
    d["newest_ymd"] = x["sales_ymd"].max()
    return pd.Series(d)

df_receipt.groupby("customer_id").apply(def_grp).reset_index().head(10)

CPU times: user 3.37 s, sys: 160 ms, total: 3.53 s
Wall time: 3.34 s


,customer_id,newest_ymd
0,CS001113000004,20190308
1,CS001114000005,20190731
2,CS001115000010,20190405
3,CS001205000004,20190625
4,CS001205000006,20190224
5,CS001211000025,20190322
6,CS001212000027,20170127
7,CS001212000031,20180906
8,CS001212000046,20170811
9,CS001212000070,20191018


**（松尾コメント）**\
・単列の場合はgroupbyにaggを使用しないので、applyにこだわる必要はない（実行時間も増加する）\
・ただ今後、集約対象の列が増えた場合の**拡張性**を考えると時と場合であろう

---
> P-025: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も古い売上日（sales_ymd）を求め、10件表示せよ。

---
> P-026: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上日（sales_ymd）と古い売上日を求め、両者が異なるデータを10件表示せよ。

In [62]:
%%time
_tmp_df = df_receipt.groupby("customer_id").agg(
    
        {"sales_ymd":[max, min]}
    
).reset_index()

_tmp_df.columns = ['_'.join(pair) for pair in _tmp_df.columns]

_tmp_df.query(
    
        'sales_ymd_max != sales_ymd_min'
    
).head(10)

CPU times: user 58 ms, sys: 0 ns, total: 58 ms
Wall time: 50.1 ms


,customer_id_,sales_ymd_max,sales_ymd_min
1,CS001114000005,20190731,20180503
2,CS001115000010,20190405,20171228
3,CS001205000004,20190625,20170914
4,CS001205000006,20190224,20180207
13,CS001214000009,20190902,20170306
14,CS001214000017,20191006,20180828
16,CS001214000048,20190929,20171109
17,CS001214000052,20190617,20180208
20,CS001215000005,20181021,20170206
21,CS001215000040,20171022,20170214


## Ver1.1
・解法1（関数定義）

In [64]:
%%time
def def_grp(x):
    d={}
    d["newest_ymd"] = x["sales_ymd"].max()
    d["oldest_ymd"] = x["sales_ymd"].min()
    
    return pd.Series(d)

# 顧客ID単位に集約処理を適用
_df = df_receipt.groupby("customer_id").apply(def_grp).reset_index()

# 最新購入日と最古購入日の異なるデータ
_df[_df["newest_ymd"] != _df["oldest_ymd"]].head(10)

CPU times: user 3.19 s, sys: 66 ms, total: 3.26 s
Wall time: 3.19 s


,customer_id,newest_ymd,oldest_ymd
1,CS001114000005,20190731,20180503
2,CS001115000010,20190405,20171228
3,CS001205000004,20190625,20170914
4,CS001205000006,20190224,20180207
13,CS001214000009,20190902,20170306
14,CS001214000017,20191006,20180828
16,CS001214000048,20190929,20171109
17,CS001214000052,20190617,20180208
20,CS001215000005,20181021,20170206
21,CS001215000040,20171022,20170214


・解法2

In [69]:
%%time
_df = df_receipt.groupby("customer_id")["sales_ymd"].agg(
    
    # [("集約後列名", "集約関数"), (), ...]をaggに
    [("newest_ymd", "max"), ("oldest_ymd", "min")]
    
).reset_index()

_df[_df["newest_ymd"] != _df["oldest_ymd"]].head(10)

CPU times: user 38.8 ms, sys: 0 ns, total: 38.8 ms
Wall time: 35.3 ms


,customer_id,newest_ymd,oldest_ymd
1,CS001114000005,20190731,20180503
2,CS001115000010,20190405,20171228
3,CS001205000004,20190625,20170914
4,CS001205000006,20190224,20180207
13,CS001214000009,20190902,20170306
14,CS001214000017,20191006,20180828
16,CS001214000048,20190929,20171109
17,CS001214000052,20190617,20180208
20,CS001215000005,20181021,20170206
21,CS001215000040,20171022,20170214


**（松尾コメント）**\
・解法2が処理時間においてベスト、**集約対象が複数の場合は結局apply?**

---
> P-027: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、降順でTOP5を表示せよ。

---
> P-028: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の中央値を計算し、降順でTOP5を表示せよ。

---
> P-029: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに商品コード（product_cd）の最頻値を求めよ。

---
> P-030: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標本分散を計算し、降順でTOP5を表示せよ。

---
> P-031: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標本標準偏差を計算し、降順でTOP5を表示せよ。

---
> P-032: レシート明細データフレーム（df_receipt）の売上金額（amount）について、25％刻みでパーセンタイル値を求めよ。

---
> P-033: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、330以上のものを抽出せよ。

In [82]:
%%time
df_receipt.groupby("store_cd").amount.mean().reset_index().rename(
    
        columns={"amount":"avg_amount"}
    
    ).query(
    
        'avg_amount >= 330'
    
)

CPU times: user 25.8 ms, sys: 812 µs, total: 26.7 ms
Wall time: 23.1 ms


,store_cd,avg_amount
1,S12013,330.194130
5,S13001,348.470386
7,S13003,350.915519
8,S13004,330.943949
12,S13015,351.111960
16,S13019,330.208616
17,S13020,337.879932
28,S13052,402.867470
30,S14010,348.791262
31,S14011,335.718333


## Ver1.1

In [88]:
%%time 
# 店舗ごとに売上金額の平均値を計算
_df = df_receipt.groupby("store_cd")["amount"].agg([("avg_amt", "mean")]).reset_index()

# 330以上を抽出
_df[_df["avg_amt"] >= 330]

CPU times: user 19.1 ms, sys: 1.39 ms, total: 20.5 ms
Wall time: 17.7 ms


,store_cd,avg_amt
1,S12013,330.194130
5,S13001,348.470386
7,S13003,350.915519
8,S13004,330.943949
12,S13015,351.111960
16,S13019,330.208616
17,S13020,337.879932
28,S13052,402.867470
30,S14010,348.791262
31,S14011,335.718333


**（松尾コメント）**\
・処理は2行になったが、「rename不要」「処理時間短縮」を実現

---
> P-034: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求めよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。


In [89]:
%%time
# 顧客単位に売上金額合計を計算
_tmp_df = df_receipt.groupby("customer_id").amount.sum().reset_index().rename(
    
        columns={"amount":"sum_amount"}

    # 顧客IDが、Zから始まる先を削除
    ).query(
    
        'customer_id.str.contains("^[^Z]", regex=True)', engine="python"
    
    )

# 全顧客の平均値を計算
_tmp_df.sum_amount.mean()

CPU times: user 42.7 ms, sys: 0 ns, total: 42.7 ms
Wall time: 39.9 ms


2547.742234529256

In [113]:
%%time
# もっと単純に
df_receipt.query(
    
        'customer_id.str.contains("^[^Z]", regex=True)', engine="python"
    
    ).groupby("customer_id").amount.sum().mean()

CPU times: user 98.3 ms, sys: 704 µs, total: 99 ms
Wall time: 95.2 ms


2547.742234529256

## Ver1.1

In [97]:
%%time
df_receipt[
    
    # 非会員を除外
    ~df_receipt["customer_id"].str.startswith("Z")
    
].groupby("customer_id")["amount"].agg([("sum_amt", "sum")]).mean()

CPU times: user 81.3 ms, sys: 0 ns, total: 81.3 ms
Wall time: 77.4 ms


sum_amt    2547.742235
dtype: float64

In [99]:
%%time
df_receipt[
    
    # 非会員を除外
    df_receipt["customer_id"].str.contains("^[^Z]", regex=True)

].groupby("customer_id")["amount"].agg([("sum_amt", "sum")]).mean()

CPU times: user 100 ms, sys: 0 ns, total: 100 ms
Wall time: 96.5 ms


sum_amt    2547.742235
dtype: float64

In [100]:
%%time
df_receipt[
    
    # 非会員を除外
    ~df_receipt["customer_id"].str.startswith("Z")
    
].groupby("customer_id").amount.sum().mean()

CPU times: user 75.8 ms, sys: 449 µs, total: 76.2 ms
Wall time: 72.7 ms


2547.742234529256

---
> P-035: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、データは10件だけ表示させれば良い。

In [111]:
%%time
# 顧客合計を計算
_group = df_receipt.groupby("customer_id").amount.sum().reset_index().rename(
    
        columns={"amount":"sum_amount"}
    
)
# 全体平均値を計算
_mean = _group.query(
    
        # Zから始まる顧客を削除 
        'customer_id.str.contains("^[^Z]", regex=True)', engine="python"
    
    ).sum_amount.mean()

# 平均値以上を抽出
# print("\n平均値:{}\n".format(_mean))
_group[_group["sum_amount"] >= _mean].head(10)

CPU times: user 36.4 ms, sys: 9.66 ms, total: 46 ms
Wall time: 39.8 ms


,customer_id,sum_amount
2,CS001115000010,3044
4,CS001205000006,3337
13,CS001214000009,4685
14,CS001214000017,4132
17,CS001214000052,5639
21,CS001215000040,3496
30,CS001304000006,3726
32,CS001305000005,3485
33,CS001305000011,4370
53,CS001315000180,3300


## Ver1.1

In [112]:
%%time 
# 顧客ごとに売上金額合計を計算
_df = df_receipt[
    
    # 非会員を除外
    ~df_receipt["customer_id"].str.startswith("Z")

# 売上金額合計を計算
].groupby("customer_id")["amount"].sum()

# 平均値以上を抽出
_df[_df > _df.mean()].to_frame().head(10)

CPU times: user 80.6 ms, sys: 1.11 ms, total: 81.7 ms
Wall time: 76 ms


,amount
customer_id,
CS001115000010,3044
CS001205000006,3337
CS001214000009,4685
CS001214000017,4132
CS001214000052,5639
CS001215000040,3496
CS001304000006,3726
CS001305000005,3485
CS001305000011,4370


**（松尾コメント）**\
・処理時間の違いは...\
「元のデータがトランザクションであり、集約した後で除外するほうが、レコード数が少なくて楽なだけ」\
「集約処理と集約後除外するレコード数次第だと思われる」\
⇒軸データ（会員のみ）に左外部結合（非会員除外）するのが**プロジェクト全体の処理**としては良い

---
> P-036: レシート明細データフレーム（df_receipt）と店舗データフレーム（df_store）を内部結合し、レシート明細データフレームの全項目と店舗データフレームの店舗名（store_name）を10件表示させよ。

---
> P-037: 商品データフレーム（df_product）とカテゴリデータフレーム（df_category）を内部結合し、商品データフレームの全項目とカテゴリデータフレームの小区分名（category_small_name）を10件表示させよ。

---
> P-038: 顧客データフレーム（df_customer）とレシート明細データフレーム（df_receipt）から、各顧客ごとの売上金額合計を求めよ。ただし、買い物の実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが'Z'から始まるもの）は除外すること。なお、結果は10件だけ表示させれば良い。

In [7]:
%%time
# 顧客データフレームの加工
_cus = df_customer[["customer_id", "gender_cd"]].query(
    
        # 女性 & 会員
        'gender_cd == "1" & customer_id.str.contains("^[^Z]", regex=True)', engine="python"
    
)
_cus.shape

CPU times: user 33.9 ms, sys: 0 ns, total: 33.9 ms
Wall time: 30.1 ms


(17918, 2)

In [8]:
%%time
# レシート明細データフレームの加工
_rec = df_receipt.groupby("customer_id").amount.sum().reset_index().rename(
        
        columns={"amount":"sum_amount"}
    
    )
_rec.shape

CPU times: user 36.2 ms, sys: 0 ns, total: 36.2 ms
Wall time: 33.1 ms


(8307, 2)

In [9]:
%%time
# データ結合
pd.merge(
    
    # 顧客データ
    _cus["customer_id"],

    # 決済データ
    _rec,

    # 左外部結合
    how = "left",

    # 顧客ID
    on = "customer_id"

# 決済なし（結合しなかった顧客）は0円に補完
).fillna(0).head(10)

CPU times: user 24.7 ms, sys: 0 ns, total: 24.7 ms
Wall time: 21.8 ms


,customer_id,sum_amount
0,CS021313000114,0.0
1,CS031415000172,5088.0
2,CS028811000001,0.0
3,CS001215000145,875.0
4,CS015414000103,3122.0
5,CS033513000180,868.0
6,CS035614000014,0.0
7,CS011215000048,3444.0
8,CS009413000079,0.0
9,CS040412000191,210.0


## Ver1.1

In [13]:
%%time
# 顧客情報の加工
df_customer[["customer_id", "gender_cd"]][
    
    # 女性の会員のみ
    (df_customer["gender_cd"] == "1") & (~df_customer["customer_id"].str.startswith("Z"))
    
].merge(
    
    # レシート情報の加工
    # 売上金額合計を計算
    df_receipt[["customer_id", "amount"]].groupby("customer_id")["amount"].agg([("sum_amt", "sum")]).reset_index(),
    
    # 顧客ID
    left_on="customer_id",
    right_on="customer_id",
    
    # 左外部結合
    how="left"
 
# 非結合先（利用なし）は0円
# 合計金額なので整数値に変換
).fillna({"sum_amt":0}).astype({"sum_amt":"int64"}).head(10)

CPU times: user 78.2 ms, sys: 0 ns, total: 78.2 ms
Wall time: 75.1 ms


,customer_id,gender_cd,sum_amt
0,CS021313000114,1,0
1,CS031415000172,1,5088
2,CS028811000001,1,0
3,CS001215000145,1,875
4,CS015414000103,1,3122
5,CS033513000180,1,868
6,CS035614000014,1,0
7,CS011215000048,1,3444
8,CS009413000079,1,0
9,CS040412000191,1,210


**（松尾コメント）**\
・一行で書きなおすことが可能（処理時間も若干短くなった）

---
> P-039: レシート明細データフレーム（df_receipt）から売上日数の多い顧客の上位20件と、売上金額合計の多い顧客の上位20件を抽出し、完全外部結合せよ。ただし、非会員（顧客IDが'Z'から始まるもの）は除外すること。

In [15]:
%%time
# 基礎データ
_tmp_df = df_receipt.query(
    
            # 非会員を除外
            'customer_id.str.contains("^[^Z]", regex=True)', engine="python"
    
        ).groupby(

            # 顧客×売上日単位に集約
            ["customer_id", "sales_ymd"]

        ).agg(
    
            # 合計金額
            {"amount":'sum'}
    
        ).reset_index().rename(
    
            columns={"amount":"_sum_amount"}
    
        )

CPU times: user 127 ms, sys: 0 ns, total: 127 ms
Wall time: 125 ms


In [248]:
# 売上日数の計算
_cnt = _tmp_df.groupby("customer_id").sales_ymd.count().reset_index().rename(columns={"sales_ymd":"cnt_sales"})
# ランクの計算
_cnt["rnk_cnt"] = _cnt.cnt_sales.rank(ascending=False, method="min")

# 売上日数TOP20
cnt = _cnt.query('rnk_cnt <= 20').sort_values("rnk_cnt", ascending=True)
cnt.shape

(21, 3)

In [245]:
# 合計金額の計算
_amt = _tmp_df.groupby("customer_id")._sum_amount.sum().reset_index().rename(columns={"_sum_amount":"sum_amount"})
# ランクの計算
_amt["rnk_amt"] = _amt.sum_amount.rank(ascending=False, method="min")

# 合計金額TOP20
amt = _amt.query('rnk_amt <= 20').sort_values("rnk_amt", ascending=True)
amt.shape

(20, 3)

In [250]:
# TOP20データを完全外部結合
pd.merge(
    
    # 売上日数
    cnt[["customer_id", "cnt_sales"]],
    # 売上金額
    amt[["customer_id", "sum_amount"]],
    
    # 完全外部結合
    how="outer",
    # 顧客ID
    on="customer_id"
    
)

,customer_id,cnt_sales,sum_amount
0,CS040214000008,23.0,NaN
1,CS010214000010,22.0,18585.0
2,CS015415000185,22.0,20153.0
3,CS010214000002,21.0,NaN
4,CS028415000007,21.0,19127.0
5,CS016415000141,20.0,18372.0
6,CS017415000097,20.0,23086.0
7,CS031414000051,19.0,19202.0
8,CS022515000226,19.0,NaN
9,CS039414000052,19.0,NaN


## Ver1.1

In [35]:
%%time
# 会員
_df = df_receipt[["customer_id", "sales_ymd", "amount"]][~df_receipt["customer_id"].str.startswith("Z")]

# 売上日数TOP20
_cnt = _df.drop_duplicates(["customer_id", "sales_ymd"]).groupby("customer_id")["sales_ymd"].agg([("cnt_ymd", "count")])
_cnt["rnk_cnt"] = _cnt["cnt_ymd"].rank(method="min", ascending=False).astype('int64')

# 売上金額TOP20 
_amt = _df.groupby("customer_id")["amount"].agg([("sum_amt", "sum")])
_amt["rnk_amt"] = _amt["sum_amt"].rank(method="min", ascending=False).astype('int64')

# TOP20を完全外部結合
pd.merge(
    
    # 売上日数
    _cnt[_cnt["rnk_cnt"] <= 20],
    # 売上金額
    _amt[_amt["rnk_amt"] <= 20],
    
    # 顧客ID
    left_index=True,
    right_index=True,
    
    # 完全外部結合
    how="outer"
    
).sort_values(["rnk_cnt", "rnk_amt"], ascending=True).astype('Int64')

CPU times: user 467 ms, sys: 0 ns, total: 467 ms
Wall time: 464 ms


,cnt_ymd,rnk_cnt,sum_amt,rnk_amt
customer_id,,,,
CS040214000008,23,1,<NA>,<NA>
CS015415000185,22,2,20153,2
CS010214000010,22,2,18585,6
CS028415000007,21,4,19127,4
CS010214000002,21,4,<NA>,<NA>
CS017415000097,20,6,23086,1
CS016415000141,20,6,18372,7
CS031414000051,19,8,19202,3
CS014214000023,19,8,<NA>,<NA>


**（松尾コメント）**\
np.nanはfloat型のため、int64では整数値に変換不可\
Int64で変換することで、欠損値がpd.naになり整数値に変換できる

---
> P-040: 全ての店舗と全ての商品を組み合わせると何件のデータとなるか調査したい。店舗（df_store）と商品（df_product）を直積した件数を計算せよ。

---
> P-041: レシート明細データフレーム（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、前日からの売上金額増減を計算せよ。なお、計算結果は10件表示すればよい。

In [72]:
# 集約
_df = df_receipt.groupby("sales_ymd").amount.sum().reset_index()

In [81]:
# 数値⇒文字⇒日付型に変換
from datetime import datetime as dt

def int2date(x):
    
    # 数値⇒文字列を日付に変換
    return dt.strptime(str(x), "%Y%m%d")

# 確認用
_tmp = _df.copy()

_tmp["ymd"] = _tmp["sales_ymd"].apply(int2date)

# 日付の連続性を確認
_tmp["diff"] = _tmp["ymd"].diff()

print(_tmp["diff"].unique())
print("データ数:{}".format(len(_tmp)))

[         'NaT' 86400000000000]
データ数:1034


In [74]:
# 前日差を計算
_df["diff"] = _df["amount"].diff()
_df.head(10)

,sales_ymd,amount,diff
0,20170101,33723,NaN
1,20170102,24165,-9558.0
2,20170103,27503,3338.0
3,20170104,36165,8662.0
4,20170105,37830,1665.0
5,20170106,32387,-5443.0
6,20170107,23415,-8972.0
7,20170108,24737,1322.0
8,20170109,26718,1981.0
9,20170110,20143,-6575.0


## Ver1.1

In [4]:
%%time
# 日付ごとに合計金額を計算
_df = df_receipt[["sales_ymd", "amount"]].groupby("sales_ymd")["amount"].agg([("sum_amt", "sum")])

# 前日差を計算
_df["diff"] = _df["sum_amt"].diff()

_df.head(10)

CPU times: user 7.18 ms, sys: 2.74 ms, total: 9.92 ms
Wall time: 7.96 ms


,sum_amt,diff
sales_ymd,,
20170101,33723,NaN
20170102,24165,-9558.0
20170103,27503,3338.0
20170104,36165,8662.0
20170105,37830,1665.0
20170106,32387,-5443.0
20170107,23415,-8972.0
20170108,24737,1322.0
20170109,26718,1981.0


---
> P-042: レシート明細データフレーム（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、各日付のデータに対し、１日前、２日前、３日前のデータを結合せよ。結果は10件表示すればよい。

In [141]:
_df = df_receipt.groupby("sales_ymd").amount.sum().reset_index()

In [143]:
pd.concat(
    
    [_df]+

    [
        # 1行~3行ずらし
        _df.shift(periods=i, axis=0).rename(

            columns={
                "sales_ymd":"sales_ymd_"+str(i),
                "amount":"amount_"+str(i)
            }
        )
        for i in range(1,4)
    ],
    
    axis=1
    
).head(10)

,sales_ymd,amount,sales_ymd_1,amount_1,sales_ymd_2,amount_2,sales_ymd_3,amount_3
0,20170101,33723,NaN,NaN,NaN,NaN,NaN,NaN
1,20170102,24165,20170101.0,33723.0,NaN,NaN,NaN,NaN
2,20170103,27503,20170102.0,24165.0,20170101.0,33723.0,NaN,NaN
3,20170104,36165,20170103.0,27503.0,20170102.0,24165.0,20170101.0,33723.0
4,20170105,37830,20170104.0,36165.0,20170103.0,27503.0,20170102.0,24165.0
5,20170106,32387,20170105.0,37830.0,20170104.0,36165.0,20170103.0,27503.0
6,20170107,23415,20170106.0,32387.0,20170105.0,37830.0,20170104.0,36165.0
7,20170108,24737,20170107.0,23415.0,20170106.0,32387.0,20170105.0,37830.0
8,20170109,26718,20170108.0,24737.0,20170107.0,23415.0,20170106.0,32387.0
9,20170110,20143,20170109.0,26718.0,20170108.0,24737.0,20170107.0,23415.0


## Ver1.1

In [104]:
%%time
# 日付ごとに合計金額を計算
_df = df_receipt[["sales_ymd", "amount"]].groupby("sales_ymd")["amount"].agg([("sum_amt", "sum")])

# 前日ずらしで横結合
_sht = pd.concat(
    # 元データ
    [_df] + [
        
        # 1～3日ずらし
        _df.shift(i, axis=0).rename(columns={"sum_amt":"sum_amt_%s" % i}).astype('Int64') for i in range(1, 4)
        
    ], axis=1
)

_sht.head(10)

CPU times: user 3.09 ms, sys: 10.8 ms, total: 13.8 ms
Wall time: 12.4 ms


,sum_amt,sum_amt_1,sum_amt_2,sum_amt_3
sales_ymd,,,,
20170101,33723,<NA>,<NA>,<NA>
20170102,24165,33723,<NA>,<NA>
20170103,27503,24165,33723,<NA>
20170104,36165,27503,24165,33723
20170105,37830,36165,27503,24165
20170106,32387,37830,36165,27503
20170107,23415,32387,37830,36165
20170108,24737,23415,32387,37830
20170109,26718,24737,23415,32387


---
> P-043： レシート明細データフレーム（df_receipt）と顧客データフレーム（df_customer）を結合し、性別（gender）と年代（ageから計算）ごとに売上金額（amount）を合計した売上サマリデータフレーム（df_sales_summary）を作成せよ。性別は0が男性、1が女性、9が不明を表すものとする。
>
> ただし、項目構成は年代、女性の売上金額、男性の売上金額、性別不明の売上金額の4項目とすること（縦に年代、横に性別のクロス集計）。また、年代は10歳ごとの階級とすること。

In [145]:
# レシート情報の加工
# 顧客単位に売上金額合計を計算
_rec = df_receipt.groupby("customer_id").amount.sum().reset_index().rename(columns={"amount":"sum_amount"})

In [146]:
# 顧客情報の加工
# 年代を計算
_cus = df_customer[["customer_id", "gender", "age"]].copy()
_cus["kbn_age"] = _cus["age"].apply(lambda x:
                                    
                                        str(int(x/10)*10) if int(x/10) > 0 else "10未満"
                                   )
# 確認用
# _cus.groupby(["kbn_age", "age"]).count().to_csv("chk.csv")

# 性別区分を作成
_cus["kbn_sex"] = _cus["gender"].apply(lambda x: 
                                       
                                       0 if x == "男性"
                                       
                                       else 1 if x == "女性"
                                       
                                       else 9
                                    )

# 確認用
# _cus.groupby(["gender", "kbn_sex"]).count()

In [147]:
# データ結合
_df = pd.merge(
    
        # 顧客情報
        _cus[["customer_id", "kbn_sex", "kbn_age"]], 
        # レシート情報
        _rec[["customer_id", "sum_amount"]],
        # 左外部結合
        how="left",
        # 顧客ID
        on="customer_id"

    # 利用なし先を0円補完
    ).fillna(0)
_df

,customer_id,kbn_sex,kbn_age,sum_amount
0,CS021313000114,1,30,0.0
1,CS037613000071,9,60,0.0
2,CS031415000172,1,40,5088.0
3,CS028811000001,1,80,0.0
4,CS001215000145,1,20,875.0
...,...,...,...,...
21966,CS002512000474,1,50,0.0
21967,CS029414000065,1,40,7935.0
21968,CS012403000043,0,40,0.0
21969,CS033512000184,1,50,0.0


In [148]:
# クロス集計
df_sales_summary = pd.pivot_table(
                        # 対象のDataFrame
                        _df, 
                        # 行
                        index="kbn_age", 
                        # 列
                        columns="kbn_sex", 
                        # 集計対象
                        values="sum_amount", 
                        # 集計対象の集計方法
                        aggfunc=sum
).reset_index()

df_sales_summary.columns = ["era", "male", "female", "unknown"]
df_sales_summary

,era,male,female,unknown
0,10,1591.0,149836.0,4317.0
1,20,72940.0,1363724.0,44328.0
2,30,177322.0,693047.0,50441.0
3,40,19355.0,9320791.0,483512.0
4,50,54320.0,6685192.0,342923.0
5,60,272469.0,987741.0,71418.0
6,70,13435.0,29764.0,2427.0
7,80,46360.0,262923.0,5111.0
8,90,0.0,6260.0,NaN


## Ver1.1
>P-043： レシート明細データフレーム（df_receipt）と顧客データフレーム（df_customer）を結合し、性別（gender）と年代（ageから計算）ごとに売上金額（amount）を合計した売上サマリデータフレーム（df_sales_summary）を作成せよ。性別は0が男性、1が女性、9が不明を表すものとする。

>ただし、項目構成は年代、女性の売上金額、男性の売上金額、性別不明の売上金額の4項目とすること（縦に年代、横に性別のクロス集計）。また、年代は10歳ごとの階級とすること。

In [40]:
# 顧客情報の加工
_cus = df_customer[["customer_id", "gender", "age"]].copy()
# 年代の計算（以上未満）
_cus["kbn_age"] = pd.cut(df_customer["age"], [i*10 for i in range(1, 10)]+[np.inf], right=False, labels=["%s代" % (i*10) for i in range(1, 10)])


# データ結合
_df = pd.merge(
    
    # 顧客情報
    _cus,
    # レシート情報
    df_receipt[["customer_id", "amount"]].groupby("customer_id")["amount"].agg([("sum_amt", "sum")]),
    
    # 顧客ID
    on="customer_id",
    # 左外部結合
    how="left"
 
# 購入実績なし先を0円補完
).fillna({"sum_amt":0}).astype({"sum_amt":"Int64"})


# クロス集計
df_sales_summary = pd.pivot_table(_df, index="kbn_age", columns="gender", values="sum_amt", aggfunc="sum")
df_sales_summary

gender,不明,女性,男性
kbn_age,,,
10代,4317,149836,1591
20代,44328,1363724,72940
30代,50441,693047,177322
40代,483512,9320791,19355
50代,342923,6685192,54320
60代,71418,987741,272469
70代,2427,29764,13435
80代,5111,262923,46360
90代,0,6260,0


---
> P-044： 前設問で作成した売上サマリデータフレーム（df_sales_summary）は性別の売上を横持ちさせたものであった。このデータフレームから性別を縦持ちさせ、年代、性別コード、売上金額の3項目に変換せよ。ただし、性別コードは男性を'00'、女性を'01'、不明を'99'とする。

In [168]:
# 初期化
_df = pd.DataFrame()
dic = {"male":"00", "female":"01", "unknown":"99"}

for sex in dic.keys():
    
    # データを抽出
    _tmp = df_sales_summary[["era", sex]].rename(columns={sex:"sum_amount"}).copy()
    
    # 性別コード
    _tmp["gender"] = dic[sex]
    
    # データを上書き
    _df = pd.concat([_df, _tmp], axis=0)
    
_df.head()

,era,sum_amount,gender
0,10,1591.0,00
1,20,72940.0,00
2,30,177322.0,00
3,40,19355.0,00
4,50,54320.0,00


In [172]:
# もっと単純に
df_sales_summary.set_index("era").stack().reset_index().replace(
    
        {"male":"00", "female":"01", "unknown":"99"}
    
    ).rename(
    
        columns={"level_1":"gender", 0:"sum_amount"}

)

,era,gender,sum_amount
0,10,00,1591.0
1,10,01,149836.0
2,10,99,4317.0
3,20,00,72940.0
4,20,01,1363724.0
5,20,99,44328.0
6,30,00,177322.0
7,30,01,693047.0
8,30,99,50441.0
9,40,00,19355.0


## Ver1.1

In [41]:
# 横持ちを縦持ちに変換
df_sales_summary.stack().reset_index().rename(columns={0:"sum_amt"}).replace(
    {"不明":"99", "女性":"01", "男性":"00"}
)

,kbn_age,gender,sum_amt
0,10代,99,4317
1,10代,01,149836
2,10代,00,1591
3,20代,99,44328
4,20代,01,1363724
5,20代,00,72940
6,30代,99,50441
7,30代,01,693047
8,30代,00,177322
9,40代,99,483512


---
> P-045: 顧客データフレーム（df_customer）の生年月日（birth_day）は日付型（Date）でデータを保有している。これをYYYYMMDD形式の文字列に変換し、顧客ID（customer_id）とともに抽出せよ。データは10件を抽出すれば良い。

In [182]:
_df = df_customer[["customer_id", "birth_day"]].copy()

In [183]:
from datetime import datetime as dt
def date2str(x):
    
    return dt.strftime(x,'%Y%m%d')

_df["ymd"] = _df["birth_day"].apply(date2str)
_df[["customer_id", "ymd"]].head(10)

,customer_id,ymd
0,CS021313000114,19810429
1,CS037613000071,19520401
2,CS031415000172,19761004
3,CS028811000001,19330327
4,CS001215000145,19950329
5,CS020401000016,19740915
6,CS015414000103,19770809
7,CS029403000008,19730817
8,CS015804000004,19310502
9,CS033513000180,19620711


## Ver1.1

In [46]:
# 顧客情報
_df = df_customer[["customer_id", "birth_day"]].copy()

# 日付を文字列に変換
_df["ymd"] = _df["birth_day"].apply(lambda x : datetime.strftime(x, "%Y%m%d"))
_df.head(10)

,customer_id,birth_day,ymd
0,CS021313000114,1981-04-29,19810429
1,CS037613000071,1952-04-01,19520401
2,CS031415000172,1976-10-04,19761004
3,CS028811000001,1933-03-27,19330327
4,CS001215000145,1995-03-29,19950329
5,CS020401000016,1974-09-15,19740915
6,CS015414000103,1977-08-09,19770809
7,CS029403000008,1973-08-17,19730817
8,CS015804000004,1931-05-02,19310502
9,CS033513000180,1962-07-11,19620711


---
> P-046: 顧客データフレーム（df_customer）の申し込み日（application_date）はYYYYMMDD形式の文字列型でデータを保有している。これを日付型（dateやdatetime）に変換し、顧客ID（customer_id）とともに抽出せよ。データは10件を抽出すれば良い。

In [184]:
from datetime import datetime as dt

_df = df_customer[["customer_id", "application_date"]].copy()

# 文字列⇒日付
def str2date(x):
    return dt.strptime(x, '%Y%m%d')

_df["ymd"] = _df["application_date"].apply(str2date)
_df[["customer_id", "ymd"]].head(10)

,customer_id,ymd
0,CS021313000114,2015-09-05
1,CS037613000071,2015-04-14
2,CS031415000172,2015-05-29
3,CS028811000001,2016-01-15
4,CS001215000145,2017-06-05
5,CS020401000016,2015-02-25
6,CS015414000103,2015-07-22
7,CS029403000008,2015-05-15
8,CS015804000004,2015-06-07
9,CS033513000180,2015-07-28


## Ver1.1

In [50]:
%%time
# 顧客情報のコピー
_df = df_customer[["customer_id", "application_date"]].copy()

# 文字列⇒日付型に変換
_df["ymd"] = pd.to_datetime(_df["application_date"])
_df.head(10)

CPU times: user 13.2 ms, sys: 650 µs, total: 13.9 ms
Wall time: 11.6 ms


,customer_id,application_date,ymd
0,CS021313000114,20150905,2015-09-05
1,CS037613000071,20150414,2015-04-14
2,CS031415000172,20150529,2015-05-29
3,CS028811000001,20160115,2016-01-15
4,CS001215000145,20170605,2017-06-05
5,CS020401000016,20150225,2015-02-25
6,CS015414000103,20150722,2015-07-22
7,CS029403000008,20150515,2015-05-15
8,CS015804000004,20150607,2015-06-07
9,CS033513000180,20150728,2015-07-28


**（松尾コメント）**\
1行で処理するなら

In [51]:
%%time
pd.concat([df_customer["customer_id"], pd.to_datetime(df_customer["application_date"])], axis=1).head(10)

CPU times: user 2.23 ms, sys: 8.91 ms, total: 11.1 ms
Wall time: 9.29 ms


,customer_id,application_date
0,CS021313000114,2015-09-05
1,CS037613000071,2015-04-14
2,CS031415000172,2015-05-29
3,CS028811000001,2016-01-15
4,CS001215000145,2017-06-05
5,CS020401000016,2015-02-25
6,CS015414000103,2015-07-22
7,CS029403000008,2015-05-15
8,CS015804000004,2015-06-07
9,CS033513000180,2015-07-28


---
> P-047: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）はYYYYMMDD形式の数値型でデータを保有している。これを日付型（dateやdatetime）に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [191]:
%%time
from datetime import datetime as dt
def int2date(x):
    return dt.strptime(str(x), '%Y%m%d')

_df = df_receipt[["receipt_no", "receipt_sub_no", "sales_ymd"]].copy()

_df["ymd"] = _df["sales_ymd"].apply(int2date)
_df.drop("sales_ymd", axis=1).head(10)

CPU times: user 1.24 s, sys: 0 ns, total: 1.24 s
Wall time: 1.24 s


,receipt_no,receipt_sub_no,ymd
0,112,1,2018-11-03
1,1132,2,2018-11-18
2,1102,1,2017-07-12
3,1132,1,2019-02-05
4,1102,2,2018-08-21
5,1112,1,2019-06-05
6,1102,2,2018-12-05
7,1102,1,2019-09-22
8,1112,2,2017-05-04
9,1102,1,2019-10-10


In [193]:
%%time
_df = pd.concat([
    
        df_receipt[["receipt_no", "receipt_sub_no"]],
    
        pd.to_datetime(df_receipt["sales_ymd"].astype('str'))
    
        ], axis=1
)
_df.head(10)

# applyより処理速度が速い

CPU times: user 152 ms, sys: 0 ns, total: 152 ms
Wall time: 149 ms


,receipt_no,receipt_sub_no,sales_ymd
0,112,1,2018-11-03
1,1132,2,2018-11-18
2,1102,1,2017-07-12
3,1132,1,2019-02-05
4,1102,2,2018-08-21
5,1112,1,2019-06-05
6,1102,2,2018-12-05
7,1102,1,2019-09-22
8,1112,2,2017-05-04
9,1102,1,2019-10-10


---
> P-048: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）は数値型のUNIX秒でデータを保有している。これを日付型（dateやdatetime）に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [199]:
%%time
_df = pd.concat(
    
        [
            df_receipt[["receipt_no", "receipt_sub_no"]],
            pd.to_datetime(df_receipt["sales_epoch"], unit="s")
        ], 
        axis=1
)
_df.head(10)

CPU times: user 7.09 ms, sys: 0 ns, total: 7.09 ms
Wall time: 5.86 ms


,receipt_no,receipt_sub_no,sales_epoch
0,112,1,2018-11-03
1,1132,2,2018-11-18
2,1102,1,2017-07-12
3,1132,1,2019-02-05
4,1102,2,2018-08-21
5,1112,1,2019-06-05
6,1102,2,2018-12-05
7,1102,1,2019-09-22
8,1112,2,2017-05-04
9,1102,1,2019-10-10


---
> P-049: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"年"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [205]:
_df = pd.concat(
    [
        df_receipt[["receipt_no", "receipt_sub_no"]],

        # 秒数⇒日付⇒dtのyearで年月を抽出
        pd.to_datetime(

            df_receipt["sales_epoch"], unit="s"

        ).dt.year
        
    ], axis=1
)
_df.head(10)

,receipt_no,receipt_sub_no,sales_epoch
0,112,1,2018
1,1132,2,2018
2,1102,1,2017
3,1132,1,2019
4,1102,2,2018
5,1112,1,2019
6,1102,2,2018
7,1102,1,2019
8,1112,2,2017
9,1102,1,2019


---
> P-050: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"月"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。なお、"月"は0埋め2桁で取り出すこと。データは10件を抽出すれば良い。

In [212]:
_df = pd.concat(
    [
        df_receipt[["receipt_no", "receipt_sub_no"]],
        
        pd.to_datetime(
            
            df_receipt["sales_epoch"], unit="s"
            
        # 日付型から文字列として月を抽出
        # dt.monthでも抽出可（0埋めにはならない）
        ).dt.strftime('%m')
        
    ], axis=1
)
_df.head(10)

,receipt_no,receipt_sub_no,sales_epoch
0,112,1,11
1,1132,2,11
2,1102,1,07
3,1132,1,02
4,1102,2,08
5,1112,1,06
6,1102,2,12
7,1102,1,09
8,1112,2,05
9,1102,1,10


---
> P-051: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"日"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。なお、"日"は0埋め2桁で取り出すこと。データは10件を抽出すれば良い。

In [211]:
_df = pd.concat(
    [
        df_receipt[["receipt_no", "receipt_sub_no"]],
        
        pd.to_datetime(
          
            # 日付型に変換
            df_receipt["sales_epoch"], unit="s"
            
        ).dt.strftime('%d')
    
    ], axis=1
)
_df.head(10)

,receipt_no,receipt_sub_no,sales_epoch
0,112,1,03
1,1132,2,18
2,1102,1,12
3,1132,1,05
4,1102,2,21
5,1112,1,05
6,1102,2,05
7,1102,1,22
8,1112,2,04
9,1102,1,10


---
> P-052: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計の上、売上金額合計に対して2000円以下を0、2000円超を1に2値化し、顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [54]:
%%time
_df = df_receipt.query(
    
        'customer_id.str.contains("^[^Z]", regex=True)', engine="python"
    
    ).groupby("customer_id").amount.sum().reset_index()

CPU times: user 103 ms, sys: 0 ns, total: 103 ms
Wall time: 99.9 ms


In [226]:
%%time
# applyを使用する場合
_df["flg_amt"] = _df["amount"].apply(lambda x: 0 if x <= 2000 else 1)
_df.head()

CPU times: user 8.33 ms, sys: 849 µs, total: 9.18 ms
Wall time: 7.21 ms


,customer_id,amount,flg_amt
0,CS001113000004,1298,0
1,CS001114000005,626,0
2,CS001115000010,3044,1
3,CS001205000004,1988,0
4,CS001205000006,3337,1


In [227]:
%time
# np.whereを使用する場合
_df["flg_amt"] = np.where(_df["amount"] <= 2000, 0, 1)
_df.head()

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.72 µs


,customer_id,amount,flg_amt
0,CS001113000004,1298,0
1,CS001114000005,626,0
2,CS001115000010,3044,1
3,CS001205000004,1988,0
4,CS001205000006,3337,1


## Ver1.1

In [56]:
%%time
# 売上金額合計を計算
_df = df_receipt[~df_receipt["customer_id"].str.startswith("Z")].groupby("customer_id")["amount"].agg([("sum_amt", "sum")])

CPU times: user 79.1 ms, sys: 0 ns, total: 79.1 ms
Wall time: 77.1 ms


In [59]:
%%time
_df["flg_amt"] = pd.cut(_df["sum_amt"], [-np.inf, 2000, np.inf], right=True, labels=[0, 1])
_df.head()

CPU times: user 7.52 ms, sys: 0 ns, total: 7.52 ms
Wall time: 4.6 ms


,sum_amt,flg_amt
customer_id,,
CS001113000004,1298,0
CS001114000005,626,0
CS001115000010,3044,1
CS001205000004,1988,0
CS001205000006,3337,1


**（松尾コメント）**\
**np.where()** を使うのが処理時間・可読性ベスト

---
> P-053: 顧客データフレーム（df_customer）の郵便番号（postal_cd）に対し、東京（先頭3桁が100〜209のもの）を1、それ以外のものを0に２値化せよ。さらにレシート明細データフレーム（df_receipt）と結合し、全期間において買い物実績のある顧客数を、作成した2値ごとにカウントせよ。

In [87]:
%%time
# 顧客データの加工
_cus = df_customer[["customer_id", "postal_cd"]].copy()

_cus["flg_tokyo"] = _cus["postal_cd"].apply(lambda x: 1 if 100 <= int(x[0:3]) <= 209 else 0)

CPU times: user 25.8 ms, sys: 1.83 ms, total: 27.6 ms
Wall time: 25.1 ms


In [88]:
%%time
# データ結合
pd.merge(
    
    _cus[["customer_id", "flg_tokyo"]],
    
    df_receipt["customer_id"],
    
    # 顧客情報と買い物実績がある先
    how="inner",
    on="customer_id"
    
).groupby("flg_tokyo").agg(
 
    # 顧客数をカウント（複数買い物実績がある人も1にカウント）
    {"customer_id":"nunique"}
    
).reset_index()

CPU times: user 30.8 ms, sys: 20.7 ms, total: 51.5 ms
Wall time: 52 ms


,flg_tokyo,customer_id
0,0,3906
1,1,4400


## Ver1.1

In [161]:
# 顧客情報の加工
_cus = df_customer[["customer_id", "postal_cd"]].copy()

# 東京の住所フラグを付与
_cus["flg_tky"] = _cus["postal_cd"].str.match(r'^(1[0-9][0-9]|20[0-9]).*')

In [162]:
%%time
# 買い物実績を結合
_df = _cus.merge(
    # 買い物実績
    df_receipt["customer_id"].drop_duplicates(),
    # 顧客ID
    on="customer_id",
    # 左外部
    how="left",
    
    # 存在有無
    indicator=True
)

CPU times: user 42.2 ms, sys: 63 µs, total: 42.3 ms
Wall time: 39.9 ms


**2値データへの置換方法案**

In [165]:
%%time
# 買い物実績有無フラグ
_df["flg_buy"] = np.where(_df["_merge"]=='both', 1, 0)

CPU times: user 3.69 ms, sys: 167 µs, total: 3.86 ms
Wall time: 2.51 ms


In [181]:
%%time
# 買い物実績有無フラグ
_df["flg_buy"].where(_df["_merge"]=='both', 0, inplace=True)

CPU times: user 3.2 ms, sys: 199 µs, total: 3.4 ms
Wall time: 1.98 ms


In [96]:
%%time
# 買い物実績有無フラグ
_df["flg_buy"] = _df["_merge"].replace({'both':1, 'left_only':0})

CPU times: user 13.1 ms, sys: 323 µs, total: 13.4 ms
Wall time: 11.6 ms


**np.where()** のほうが早い\
Series.where()のほうが速いが、値を明示的に指定できないので今回のような処理では使いたくない

In [97]:
# クロス主計（合計も表示 : margins）
pd.pivot_table(_df, index="flg_tky", columns="flg_buy", values="customer_id", aggfunc="count", margins=True)

flg_buy,0,1,All
flg_tky,,,
False,6137,3906,10043
True,7528,4400,11928
All,13665,8306,21971


**（松尾コメント）**\
・2値変数への変換にはnp.where()を使用し、高速化\
・pd.pivot_table()を使用して集計実施

---
> P-054: 顧客データデータフレーム（df_customer）の住所（address）は、埼玉県、千葉県、東京都、神奈川県のいずれかとなっている。都道府県毎にコード値を作成し、顧客ID、住所とともに抽出せよ。値は埼玉県を11、千葉県を12、東京都を13、神奈川県を14とすること。結果は10件表示させれば良い。

In [254]:
_df = df_customer[["customer_id", "address"]].copy()

# 変換辞書リスト
dct = {"埼玉県":"11", "千葉県":"12", "東京都":"13", "神奈川":"14"}

In [252]:
%%time
# 辞書検索関数
def chk_dct(x):
    for i in dct.keys():
        if x.startswith(i):
            return dct[i]

_df["kbn_address"] = _df["address"].apply(chk_dct)
_df.head(10)

CPU times: user 24.2 ms, sys: 323 µs, total: 24.5 ms
Wall time: 22.1 ms


,customer_id,address,kbn_address
0,CS021313000114,神奈川県伊勢原市粟窪**********,14
1,CS037613000071,東京都江東区南砂**********,13
2,CS031415000172,東京都渋谷区代々木**********,13
3,CS028811000001,神奈川県横浜市泉区和泉町**********,14
4,CS001215000145,東京都大田区仲六郷**********,13
5,CS020401000016,東京都板橋区若木**********,13
6,CS015414000103,東京都江東区北砂**********,13
7,CS029403000008,千葉県浦安市海楽**********,12
8,CS015804000004,東京都江東区北砂**********,13
9,CS033513000180,神奈川県横浜市旭区善部町**********,14


In [257]:
%%time
_tmp = pd.concat(
    [_df, _df["address"].str[0:3].map(dct)], axis=1
)
_tmp.head(10)

CPU times: user 14.9 ms, sys: 440 µs, total: 15.3 ms
Wall time: 14.2 ms


,customer_id,address,address
0,CS021313000114,神奈川県伊勢原市粟窪**********,14
1,CS037613000071,東京都江東区南砂**********,13
2,CS031415000172,東京都渋谷区代々木**********,13
3,CS028811000001,神奈川県横浜市泉区和泉町**********,14
4,CS001215000145,東京都大田区仲六郷**********,13
5,CS020401000016,東京都板橋区若木**********,13
6,CS015414000103,東京都江東区北砂**********,13
7,CS029403000008,千葉県浦安市海楽**********,12
8,CS015804000004,東京都江東区北砂**********,13
9,CS033513000180,神奈川県横浜市旭区善部町**********,14


## Ver1.1

In [106]:
# 顧客情報
_cus = df_customer[["customer_id", "address"]].copy()

# 変換辞書リスト
dct = {"埼玉県":"11", "千葉県":"12", "東京都":"13", "神奈川県":"14"}

**コード値への変換方法案**

案1：replace()

In [112]:
%%time
# 都道府県を抽出して置換
_cus["address_1"] = _cus["address"].str.extract(r'^(.*都|.*道|.*府|.*県)').replace(dct)

CPU times: user 58.7 ms, sys: 0 ns, total: 58.7 ms
Wall time: 56.6 ms


案2：apply()

In [113]:
%%time
# 都道府県を抽出して置換
_cus["address_1"] = _cus["address"].str.extract(r'^(.*都|.*道|.*府|.*県)', expand=False).apply(lambda x : dct.get(x))

CPU times: user 39.1 ms, sys: 532 µs, total: 39.6 ms
Wall time: 37.4 ms


In [114]:
_cus.head(10)

,customer_id,address,address_1
0,CS021313000114,神奈川県伊勢原市粟窪**********,14
1,CS037613000071,東京都江東区南砂**********,13
2,CS031415000172,東京都渋谷区代々木**********,13
3,CS028811000001,神奈川県横浜市泉区和泉町**********,14
4,CS001215000145,東京都大田区仲六郷**********,13
5,CS020401000016,東京都板橋区若木**********,13
6,CS015414000103,東京都江東区北砂**********,13
7,CS029403000008,千葉県浦安市海楽**********,12
8,CS015804000004,東京都江東区北砂**********,13
9,CS033513000180,神奈川県横浜市旭区善部町**********,14


**（松尾コメント）**\
・見栄えは処理内容がわかりやすいが、文字列抽出で時間がかかっている\
・文字列変換ではreplace()よりもapply()のほうが速い

---
> P-055: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、その合計金額の四分位点を求めよ。その上で、顧客ごとの売上金額合計に対して以下の基準でカテゴリ値を作成し、顧客ID、売上金額と合計ともに表示せよ。カテゴリ値は上から順に1〜4とする。結果は10件表示させれば良い。
>
> - 最小値以上第一四分位未満
> - 第一四分位以上第二四分位未満
> - 第二四分位以上第三四分位未満
> - 第三四分位以上

In [48]:
%%time
# 顧客単位で売上金額合計を計算
_df = df_receipt.groupby("customer_id").amount.sum().reset_index().rename(columns={"amount":"sum_amt"})

CPU times: user 1.03 s, sys: 207 ms, total: 1.23 s
Wall time: 1.23 s


In [49]:
%%time
# 売上金額合計の四分位点（25%, 50%, 75%）を計算
_qtl = _df.sum_amt.quantile([i/4 for i in range(1,4)]).reset_index()
# 辞書に格納
dct = {n+1:amt for n, amt in enumerate(_qtl["sum_amt"])}
dct

CPU times: user 11.7 ms, sys: 761 µs, total: 12.4 ms
Wall time: 8.85 ms


{1: 548.5, 2: 1478.0, 3: 3651.0}

In [50]:
%%time
def chk_amt(x):
    for i in dct.keys():
        if x < dct[i]:
            return i
        else:
            result = i+1
    return result
        
_df["kbn_amt"] = _df["sum_amt"].apply(chk_amt)
_df.head(10)

CPU times: user 40.8 ms, sys: 0 ns, total: 40.8 ms
Wall time: 37.4 ms


,customer_id,sum_amt,kbn_amt
0,CS001113000004,1298,2
1,CS001114000005,626,2
2,CS001115000010,3044,3
3,CS001205000004,1988,3
4,CS001205000006,3337,3
5,CS001211000025,456,1
6,CS001212000027,448,1
7,CS001212000031,296,1
8,CS001212000046,228,1
9,CS001212000070,456,1


## Ver1.1

In [51]:
%%time
# 顧客単位に売上金額合計
_df = df_receipt[["customer_id", "amount"]].groupby("customer_id")["amount"].agg([("sum_amt", "sum")])

CPU times: user 33.2 ms, sys: 2.11 ms, total: 35.3 ms
Wall time: 32.8 ms


In [52]:
%%time
# 四分位点を計算
_qtl = _df["sum_amt"].quantile([i/4 for i in range(4)])
_qtl

CPU times: user 6.43 ms, sys: 0 ns, total: 6.43 ms
Wall time: 4.11 ms


0.00      70.0
0.25     548.5
0.50    1478.0
0.75    3651.0
Name: sum_amt, dtype: float64

In [53]:
%%time
# 四分位点でカテゴライズ
_df["kbn_amt"] = pd.cut(
    
    # 対象の列
    _df["sum_amt"],
    
    # 閾値リスト
    _qtl,
    
    # 以上未満
    # ※75パーセンタイル以上は変換できていない
    right=False,
    
    # カテゴリ名
    labels=[1, 2, 3]
    
# 第3四分位以上
).fillna({"kbn_amt":4})
_df.head(10)

CPU times: user 14.8 ms, sys: 0 ns, total: 14.8 ms
Wall time: 11.4 ms


,sum_amt,kbn_amt
customer_id,,
CS001113000004,1298,2
CS001114000005,626,2
CS001115000010,3044,3
CS001205000004,1988,3
CS001205000006,3337,3
CS001211000025,456,1
CS001212000027,448,1
CS001212000031,296,1
CS001212000046,228,1


**（松尾コメント）**\
ビン化する関数がある（知らないだけで便利な関数が多い）\
※当然だがapplyを使用しないので処理も早い\
※quantile()とcut()を組み合わせたqcut()という関数もある

---
> P-056: 顧客データフレーム（df_customer）の年齢（age）をもとに10歳刻みで年代を算出し、顧客ID（customer_id）、生年月日（birth_day）とともに抽出せよ。ただし、60歳以上は全て60歳代とすること。年代を表すカテゴリ名は任意とする。先頭10件を表示させればよい。

In [60]:
%%time
_df = pd.concat(
    [
        df_customer[["customer_id", "birth_day"]],
        
        df_customer["age"].apply(lambda x: int(x/10)*10 if x < 60 else 60)
    
    ], axis=1
)
_df.head(10)

CPU times: user 23.4 ms, sys: 0 ns, total: 23.4 ms
Wall time: 20.1 ms


,customer_id,birth_day,age
0,CS021313000114,1981-04-29,30
1,CS037613000071,1952-04-01,60
2,CS031415000172,1976-10-04,40
3,CS028811000001,1933-03-27,60
4,CS001215000145,1995-03-29,20
5,CS020401000016,1974-09-15,40
6,CS015414000103,1977-08-09,40
7,CS029403000008,1973-08-17,40
8,CS015804000004,1931-05-02,60
9,CS033513000180,1962-07-11,50


## Ver1.1

In [61]:
%%time
# 必要な情報を抽出
_df = df_customer[["customer_id", "birth_day", "age"]].copy()

# 年齢区分を計算
_df["kbn_age"] = pd.cut(
    
    # ビン化対象列
    _df["age"],
    # 閾値リスト
    [i*10 for i in range(7)] + [np.inf],
    
    # 以上未満
    right=False,
    
    # ラベル名
    labels=["%s代" % (i*10) for i in range(7)]
    
)

_df.head(10)

CPU times: user 13.4 ms, sys: 0 ns, total: 13.4 ms
Wall time: 7.88 ms


,customer_id,birth_day,age,kbn_age
0,CS021313000114,1981-04-29,37,30代
1,CS037613000071,1952-04-01,66,60代
2,CS031415000172,1976-10-04,42,40代
3,CS028811000001,1933-03-27,86,60代
4,CS001215000145,1995-03-29,24,20代
5,CS020401000016,1974-09-15,44,40代
6,CS015414000103,1977-08-09,41,40代
7,CS029403000008,1973-08-17,45,40代
8,CS015804000004,1931-05-02,87,60代
9,CS033513000180,1962-07-11,56,50代


**（松尾コメント）**\
閾値にnp.infを設定することで、上限なしの区分もビン化できる\
※以下確認用

In [56]:
[i*10 for i in range(7)] + [np.inf]

[0, 10, 20, 30, 40, 50, 60, inf]

In [57]:
["%s歳代" % (i*10) for i in range(7)]

['0歳代', '10歳代', '20歳代', '30歳代', '40歳代', '50歳代', '60歳代']

---
> P-057: 前問題の抽出結果と性別（gender）を組み合わせ、新たに性別×年代の組み合わせを表すカテゴリデータを作成せよ。組み合わせを表すカテゴリの値は任意とする。先頭10件を表示させればよい。

---
> P-058: 顧客データフレーム（df_customer）の性別コード（gender_cd）をダミー変数化し、顧客ID（customer_id）とともに抽出せよ。結果は10件表示させれば良い。

---
> P-059: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を平均0、標準偏差1に標準化して顧客ID、売上金額合計とともに表示せよ。標準化に使用する標準偏差は、不偏標準偏差と標本標準偏差のどちらでも良いものとする。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [307]:
# 顧客単位に売上金額合計を計算
_df = df_receipt.query(
        'customer_id.str.contains("^[^Z]", regex=True)', engine="python"
    ).groupby(
        "customer_id"
    ).amount.sum().reset_index().rename(
        columns={"amount":"sum_amt"}
    )
_df.head()

,customer_id,sum_amt
0,CS001113000004,1298
1,CS001114000005,626
2,CS001115000010,3044
3,CS001205000004,1988
4,CS001205000006,3337


In [314]:
# 売上金額合計を分散標準化
from sklearn import preprocessing

_df["z_sum_amt"] = preprocessing.scale(_df["sum_amt"])
_df.head(10)

,customer_id,sum_amt,z_sum_amt
0,CS001113000004,1298,-0.459378
1,CS001114000005,626,-0.706390
2,CS001115000010,3044,0.182413
3,CS001205000004,1988,-0.205749
4,CS001205000006,3337,0.290114
5,CS001211000025,456,-0.768879
6,CS001212000027,448,-0.771819
7,CS001212000031,296,-0.827691
8,CS001212000046,228,-0.852686
9,CS001212000070,456,-0.768879


## Ver1.1

In [115]:
from sklearn import preprocessing

In [117]:
# 売上金額合計を計算
_df = df_receipt[~df_receipt["customer_id"].str.startswith("Z")].groupby("customer_id")["amount"].agg([("sum_amt","sum")])

# 標準化
_df["z_scr"] = preprocessing.scale(_df["sum_amt"])
_df.head(10)

,sum_amt,z_scr
customer_id,,
CS001113000004,1298,-0.459378
CS001114000005,626,-0.706390
CS001115000010,3044,0.182413
CS001205000004,1988,-0.205749
CS001205000006,3337,0.290114
CS001211000025,456,-0.768879
CS001212000027,448,-0.771819
CS001212000031,296,-0.827691
CS001212000046,228,-0.852686


---
> P-060: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を最小値0、最大値1に正規化して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [317]:
_df = df_receipt.query(
 
    'customer_id.str.contains("^[^Z]", regex=True)', engine="python"

).groupby(
    "customer_id"
).amount.sum().reset_index().rename(
    columns={"amount":"sum_amt"}
)

In [318]:
# 売上金額合計を最小最大標準化
from sklearn import preprocessing

_df["minmax_amt"] = preprocessing.minmax_scale(_df["sum_amt"])
_df.head(10)

,customer_id,sum_amt,minmax_amt
0,CS001113000004,1298,0.053354
1,CS001114000005,626,0.024157
2,CS001115000010,3044,0.129214
3,CS001205000004,1988,0.083333
4,CS001205000006,3337,0.141945
5,CS001211000025,456,0.016771
6,CS001212000027,448,0.016423
7,CS001212000031,296,0.009819
8,CS001212000046,228,0.006865
9,CS001212000070,456,0.016771


## Ver1.1

In [118]:
from sklearn import preprocessing

In [120]:
# 売上金額合計を計算
_df = df_receipt[~df_receipt["customer_id"].str.startswith("Z")].groupby("customer_id")["amount"].agg([("sum_amt", "sum")])

# 標準化（最小最大）
_df["scl"] = preprocessing.minmax_scale(_df["sum_amt"])
_df.head(10)

,sum_amt,scl
customer_id,,
CS001113000004,1298,0.053354
CS001114000005,626,0.024157
CS001115000010,3044,0.129214
CS001205000004,1988,0.083333
CS001205000006,3337,0.141945
CS001211000025,456,0.016771
CS001212000027,448,0.016423
CS001212000031,296,0.009819
CS001212000046,228,0.006865


---
> P-061: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を常用対数化（底=10）して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [320]:
_df = df_receipt.query(
    
    'customer_id.str.contains("^[^Z]", regex=True)', engine="python"
    
).groupby(
    
    "customer_id"

).amount.sum().reset_index().rename(
    
    columns={"amount":"sum_amt"}
    
)

In [321]:
_df["log_amt"] = np.log10(_df["sum_amt"])
_df.head(10)

,customer_id,sum_amt,log_amt
0,CS001113000004,1298,3.113275
1,CS001114000005,626,2.796574
2,CS001115000010,3044,3.483445
3,CS001205000004,1988,3.298416
4,CS001205000006,3337,3.523356
5,CS001211000025,456,2.658965
6,CS001212000027,448,2.651278
7,CS001212000031,296,2.471292
8,CS001212000046,228,2.357935
9,CS001212000070,456,2.658965


## Ver1.1

In [123]:
# 売上金額合計を計算
_df = df_receipt[~df_receipt["customer_id"].str.startswith("Z")][["customer_id", "amount"]].groupby("customer_id")["amount"].agg([("sum_amt", "sum")])

# 常用対数を計算
_df["log10"] = np.log10(_df["sum_amt"])
_df.head(10)

,sum_amt,log10
customer_id,,
CS001113000004,1298,3.113275
CS001114000005,626,2.796574
CS001115000010,3044,3.483445
CS001205000004,1988,3.298416
CS001205000006,3337,3.523356
CS001211000025,456,2.658965
CS001212000027,448,2.651278
CS001212000031,296,2.471292
CS001212000046,228,2.357935


---
> P-062: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を自然対数化(底=e）して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [125]:
%%time
_df = df_receipt.query(
    'customer_id.str.contains("^[^Z]", regex=True)', engine="python"
).groupby(
    "customer_id"
).amount.sum(
).reset_index(
).rename(
    columns={"amount":"sum_amt"}
)

CPU times: user 100 ms, sys: 686 µs, total: 101 ms
Wall time: 97.9 ms


In [126]:
%%time
_df["loge_amt"] = np.log1p(_df["sum_amt"])
_df.head(10)

CPU times: user 3.72 ms, sys: 413 µs, total: 4.14 ms
Wall time: 2.58 ms


,customer_id,sum_amt,loge_amt
0,CS001113000004,1298,7.169350
1,CS001114000005,626,6.440947
2,CS001115000010,3044,8.021256
3,CS001205000004,1988,7.595387
4,CS001205000006,3337,8.113127
5,CS001211000025,456,6.124683
6,CS001212000027,448,6.107023
7,CS001212000031,296,5.693732
8,CS001212000046,228,5.433722
9,CS001212000070,456,6.124683


## Ver1.1

In [127]:
%%time
# 売上金額合計を計算
_df = df_receipt[~df_receipt["customer_id"].str.startswith("Z")][["customer_id", "amount"]].groupby("customer_id")["amount"].agg([("sum_amt", "sum")])

# 事前対数を計算
_df["logE"] = np.log1p(_df["sum_amt"])
_df.head(10)

CPU times: user 82 ms, sys: 0 ns, total: 82 ms
Wall time: 78.2 ms


,sum_amt,logE
customer_id,,
CS001113000004,1298,7.169350
CS001114000005,626,6.440947
CS001115000010,3044,8.021256
CS001205000004,1988,7.595387
CS001205000006,3337,8.113127
CS001211000025,456,6.124683
CS001212000027,448,6.107023
CS001212000031,296,5.693732
CS001212000046,228,5.433722


---
> P-063: 商品データフレーム（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益額を算出せよ。結果は10件表示させれば良い。

---
> P-064: 商品データフレーム（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益率の全体平均を算出せよ。
ただし、単価と原価にはNULLが存在することに注意せよ。

In [333]:
_df = df_product.copy()

_df["unit_profit_rate"] = np.divide(

    # 利益率 = 利益額/単価
    (_df["unit_price"] - _df["unit_cost"]), _df["unit_price"]
)

# 計算結果の確認
_df[_df["unit_profit_rate"].isnull()]

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,unit_profit_rate
159,P040802007,04,0408,040802,NaN,NaN,NaN
196,P050103021,05,0501,050103,NaN,NaN,NaN
496,P050405009,05,0504,050405,NaN,NaN,NaN
1531,P060802026,06,0608,060802,NaN,NaN,NaN
2012,P070202092,07,0702,070202,NaN,NaN,NaN
6296,P080504027,08,0805,080504,NaN,NaN,NaN
7075,P090204185,09,0902,090204,NaN,NaN,NaN


In [335]:
_df["unit_profit_rate"].mean(skipna=True)

0.24911389885176904

## Ver1.1

In [132]:
# 商品情報のコピー
_df = df_product.copy()

# 利益率の計算
_df["unit_profit_rate"] = np.divide((_df["unit_price"]-_df["unit_cost"]), _df["unit_price"])

# 平均値の計算
_df["unit_profit_rate"].mean(skipna=True)

0.24911389885176904

---
> P-065: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。ただし、1円未満は切り捨てること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [9]:
_df = df_product[["product_cd", "unit_cost"]].copy()

In [26]:
%%time
# 単価の計算1-1
# 利益率 = (単価-原価)/単価　⇒　単価 = 原価/(1-利益率)
_df["new_unit_price"] = _df["unit_cost"].apply(lambda x: 
                                               # nanはfloat型なので変換できない
                                               # int(x/(1-0.3)))
                                               
                                               # np.floorの返却値は「np.float」のためnp.nanを含んだ小数点以下の切り捨てに対応
                                               np.floor(x/0.7)
                                              )

CPU times: user 19.8 ms, sys: 0 ns, total: 19.8 ms
Wall time: 17.5 ms


In [27]:
%%time
# 単価の計算1-2
# 利益率 = (単価-原価)/単価　⇒　単価 = 原価/(1-利益率)
_df["new_unit_price"] = np.floor(np.divide(_df["unit_cost"],0.7))

CPU times: user 2.64 ms, sys: 133 µs, total: 2.77 ms
Wall time: 1.66 ms


In [28]:
%%time
# 単価の計算2
def clc_unit(x):
    if np.isnan(x):
        return x
    else:
        return int(x/(1-0.3))
    
_df["new_unit_price"] = _df["unit_cost"].apply(clc_unit)

CPU times: user 27.9 ms, sys: 0 ns, total: 27.9 ms
Wall time: 26 ms


In [22]:
# 新利益率の計算
_df["new_unit_profit_rate"] = np.divide((_df["new_unit_price"] - _df["unit_cost"]), _df["new_unit_price"])
_df.head(10)

,product_cd,unit_cost,new_unit_price,new_unit_profit_rate
0,P040101001,149.0,212.0,0.297170
1,P040101002,164.0,234.0,0.299145
2,P040101003,173.0,247.0,0.299595
3,P040101004,186.0,265.0,0.298113
4,P040101005,201.0,287.0,0.299652
5,P040101006,224.0,320.0,0.300000
6,P040101007,254.0,362.0,0.298343
7,P040101008,315.0,450.0,0.300000
8,P040101009,374.0,534.0,0.299625
9,P040101010,435.0,621.0,0.299517


## Ver1.1
>P-065: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。ただし、1円未満は切り捨てること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [136]:
# 利益率
PRFT_RT = 0.3

# 商品データのコピー
_df = df_product.copy()

# 新単価計算
_df["new_price"] = np.floor(np.divide(_df["unit_cost"], (1-PRFT_RT))).astype('Int64')

# 利益率の計算
_df["new_profit_rate"] = np.divide((_df["new_price"]-_df["unit_cost"]), _df["new_price"])

_df.head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,new_price,new_profit_rate
0,P040101001,04,0401,040101,198.0,149.0,212,0.29717
1,P040101002,04,0401,040101,218.0,164.0,234,0.299145
2,P040101003,04,0401,040101,230.0,173.0,247,0.299595
3,P040101004,04,0401,040101,248.0,186.0,265,0.298113
4,P040101005,04,0401,040101,268.0,201.0,287,0.299652
5,P040101006,04,0401,040101,298.0,224.0,320,0.3
6,P040101007,04,0401,040101,338.0,254.0,362,0.298343
7,P040101008,04,0401,040101,420.0,315.0,450,0.3
8,P040101009,04,0401,040101,498.0,374.0,534,0.299625
9,P040101010,04,0401,040101,580.0,435.0,621,0.299517


---
> P-066: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を四捨五入すること（0.5については偶数方向の丸めで良い）。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [29]:
_df = df_product[["product_cd", "unit_cost"]].copy()

In [30]:
%%time
# 新単価の計算
_df["new_unit_price"] = _df["unit_cost"].apply(lambda x: round(x/0.7, 0))

CPU times: user 13.8 ms, sys: 0 ns, total: 13.8 ms
Wall time: 12.2 ms


In [31]:
%%time
_df["new_unit_price"] = round(np.divide(_df["unit_cost"], 0.7), 0)

CPU times: user 2.41 ms, sys: 710 µs, total: 3.12 ms
Wall time: 2.2 ms


In [32]:
# 新利益率の確認
_df["new_profit_rate"] = np.divide((_df["new_unit_price"] - _df["unit_cost"]), _df["new_unit_price"])
_df.head(10)

,product_cd,unit_cost,new_unit_price,new_profit_rate
0,P040101001,149.0,213.0,0.300469
1,P040101002,164.0,234.0,0.299145
2,P040101003,173.0,247.0,0.299595
3,P040101004,186.0,266.0,0.300752
4,P040101005,201.0,287.0,0.299652
5,P040101006,224.0,320.0,0.300000
6,P040101007,254.0,363.0,0.300275
7,P040101008,315.0,450.0,0.300000
8,P040101009,374.0,534.0,0.299625
9,P040101010,435.0,621.0,0.299517


## Ver1.1
>P-066: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を四捨五入すること（0.5については偶数方向の丸めで良い）。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [137]:
# 新利益率(PRoFit-RaTe)
PRFRT = 0.3

# 商品データのコピー
_df = df_product.copy()

# 新単価の計算（小数点四捨五入）
_df["new_price"] = round(np.divide(_df["unit_cost"], (1-PRFRT)), 0).astype('Int64')

# 利益率の計算
_df["new_profit_rate"] = np.divide((_df["new_price"]-_df["unit_cost"]), _df["new_price"])

_df.head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,new_price,new_profit_rate
0,P040101001,04,0401,040101,198.0,149.0,213,0.300469
1,P040101002,04,0401,040101,218.0,164.0,234,0.299145
2,P040101003,04,0401,040101,230.0,173.0,247,0.299595
3,P040101004,04,0401,040101,248.0,186.0,266,0.300752
4,P040101005,04,0401,040101,268.0,201.0,287,0.299652
5,P040101006,04,0401,040101,298.0,224.0,320,0.3
6,P040101007,04,0401,040101,338.0,254.0,363,0.300275
7,P040101008,04,0401,040101,420.0,315.0,450,0.3
8,P040101009,04,0401,040101,498.0,374.0,534,0.299625
9,P040101010,04,0401,040101,580.0,435.0,621,0.299517


---
> P-067: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を切り上げること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [37]:
# データのコピー
_df = df_product[["product_cd", "unit_cost"]].copy()

# 新単価の計算
_df["new_unit_price"] = np.ceil(np.divide(_df["unit_cost"], 0.7))

# 新利益率の計算
_df["new_profit_rate"] = np.divide((_df["new_unit_price"] - _df["unit_cost"]), _df["new_unit_price"]) 

_df.head(10)

,product_cd,unit_cost,new_unit_price,new_profit_rate
0,P040101001,149.0,213.0,0.300469
1,P040101002,164.0,235.0,0.302128
2,P040101003,173.0,248.0,0.302419
3,P040101004,186.0,266.0,0.300752
4,P040101005,201.0,288.0,0.302083
5,P040101006,224.0,320.0,0.300000
6,P040101007,254.0,363.0,0.300275
7,P040101008,315.0,451.0,0.301552
8,P040101009,374.0,535.0,0.300935
9,P040101010,435.0,622.0,0.300643


---
> P-068: 商品データフレーム（df_product）の各商品について、消費税率10%の税込み金額を求めよ。 1円未満の端数は切り捨てとし、結果は10件表示すれば良い。ただし、単価（unit_price）にはNULLが存在することに注意せよ。

In [41]:
# データのコピー
_df = df_product.copy()

# 税込み価格の計算
_df["unit_price_intax"] = np.floor(_df["unit_price"]*1.1)

_df.head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,unit_price_intax
0,P040101001,04,0401,040101,198.0,149.0,217.0
1,P040101002,04,0401,040101,218.0,164.0,239.0
2,P040101003,04,0401,040101,230.0,173.0,253.0
3,P040101004,04,0401,040101,248.0,186.0,272.0
4,P040101005,04,0401,040101,268.0,201.0,294.0
5,P040101006,04,0401,040101,298.0,224.0,327.0
6,P040101007,04,0401,040101,338.0,254.0,371.0
7,P040101008,04,0401,040101,420.0,315.0,462.0
8,P040101009,04,0401,040101,498.0,374.0,547.0
9,P040101010,04,0401,040101,580.0,435.0,638.0


## Ver1.1
まとめると\
・小数点切り捨て：np.floor()\
・小数点切り上げ：np.ceil()\
・四捨五入：round(, 位置)

---
> P-069: レシート明細データフレーム（df_receipt）と商品データフレーム（df_product）を結合し、顧客毎に全商品の売上金額合計と、カテゴリ大区分（category_major_cd）が"07"（瓶詰缶詰）の売上金額合計を計算の上、両者の比率を求めよ。抽出対象はカテゴリ大区分"07"（瓶詰缶詰）の購入実績がある顧客のみとし、結果は10件表示させればよい。

In [54]:
# レシートデータの加工
_rec = df_receipt[
    # 顧客ID, 商品番号, 売上金額
    ["customer_id", "product_cd", "amount"]
].copy().sort_values("customer_id")
# _rec.head()

In [55]:
# 商品データの加工
_pro = df_product[
    # 商品番号, カテゴリ大区分
    ["product_cd", "category_major_cd"]
].copy()
# _pro.head()

In [49]:
# データ結合
# 内部結合 = 商品情報があるデータのみ
_df = pd.merge(_rec, _pro, how="inner", on="product_cd")

In [57]:
%%time
# 集約計算
_df_grp = pd.merge(
    # 売上金額総合計
    _df.groupby("customer_id").amount.sum().reset_index().rename(columns={"amount":"sum_amt_all"}),

    # 07売上金額合計
    _df.query('category_major_cd == "07"').groupby("customer_id").amount.sum().reset_index().rename(columns={"amount":"sum_amt_07"}),

    # 内部結合 = 購入実績のある顧客のみ
    how="inner",
    on="customer_id"
    
)

# 購入比率を計算
_df_grp["amt_rate_07"] = np.divide(_df_grp["sum_amt_07"], _df_grp["sum_amt_all"])
_df_grp.head(10)

CPU times: user 87.1 ms, sys: 11.4 ms, total: 98.4 ms
Wall time: 94.5 ms


,customer_id,sum_amt_all,sum_amt_07,amt_rate_07
0,CS001113000004,1298,1298,1.000000
1,CS001114000005,626,486,0.776358
2,CS001115000010,3044,2694,0.885020
3,CS001205000004,1988,346,0.174044
4,CS001205000006,3337,2004,0.600539
5,CS001212000027,448,200,0.446429
6,CS001212000031,296,296,1.000000
7,CS001212000046,228,108,0.473684
8,CS001212000070,456,308,0.675439
9,CS001213000018,243,145,0.596708


## Ver1.1

In [138]:
# 決済データに商品データを結合
_df = pd.merge(
    
    # 決済データ
    df_receipt[["customer_id", "amount", "product_cd"]],
    # 商品データ
    df_product[["product_cd", "category_major_cd"]],
    
    # 商品コード
    on="product_cd",
    # 内部結合
    how="inner"
    
)

In [75]:
# 瓶詰缶詰の購入実績のある顧客IDリスト
_trg = _df[_df["category_major_cd"] == "07"]["customer_id"].drop_duplicates()

In [79]:
# 集約計算
_df_grp = _df[
    
    # 対象の顧客IDリスト内の顧客のみ抽出
    _df["customer_id"].isin(_trg)
    
].groupby("customer_id").apply(
    
    lambda x : pd.Series(
        
        [
            # 売上金額合計
            x["amount"].sum(),
        
            # 大区分=07（瓶詰缶詰）
            x[x["category_major_cd"] == "07"]["amount"].sum()
         
            
        # 列名
        ], index=["sum_amt_all", "sum_amt_7"]
        
        
    )
    
)

In [80]:
# 購入金額の比率を計算
_df_grp["amt_rat_7"] = np.divide(_df_grp["sum_amt_7"], _df_grp["sum_amt_all"])
_df_grp.head(10)

,sum_amt_all,sum_amt_7,amt_rat_7
customer_id,,,
CS001113000004,1298,1298,1.000000
CS001114000005,626,486,0.776358
CS001115000010,3044,2694,0.885020
CS001205000004,1988,346,0.174044
CS001205000006,3337,2004,0.600539
CS001212000027,448,200,0.446429
CS001212000031,296,296,1.000000
CS001212000046,228,108,0.473684
CS001212000070,456,308,0.675439


**（松尾コメント）**\
groupbyとapplyを使用し、条件別集約をすることができる\
また、list型で返却できるため**内包表記でカテゴリごとの集約をまとめて処理することができる**（これでかい！！）\
※以下確認\
isin()関数が便利

In [82]:
_df[_df["customer_id"].isin(_df_grp.index[:2])].sort_values("customer_id")

,customer_id,amount,product_cd,category_major_cd
39074,CS001113000004,1200,P071401009,07
73357,CS001113000004,98,P071001012,07
1390,CS001114000005,100,P080803001,08
3561,CS001114000005,40,P050101001,05
83965,CS001114000005,338,P070101119,07
86966,CS001114000005,148,P070301002,07


## Ver1.1（応用編）
**内包表記で一括処理**

In [143]:
# 大区分のリスト抽出
_kbns = df_product["category_major_cd"].drop_duplicates()

In [144]:
%%time
# 集約計算
_tmp = _df.groupby("customer_id").apply(
    
    lambda x : pd.Series(
        
        # 売上金額合計
        [x["amount"].sum()] + 
        
        # 大区分ごと
        [x[x["category_major_cd"] == kbn]["amount"].sum() for kbn in _kbns],
           
        # 列名
        index=["sum_amt_all"] + ["sum_amt_%s" % kbn for kbn in _kbns]
        
    )
)
_tmp

CPU times: user 23.2 s, sys: 455 ms, total: 23.6 s
Wall time: 23.1 s


,sum_amt_all,sum_amt_04,sum_amt_05,sum_amt_06,sum_amt_07,sum_amt_08,sum_amt_09
customer_id,,,,,,,
CS001113000004,1298,0,0,0,1298,0,0
CS001114000005,626,0,40,0,486,100,0
CS001115000010,3044,0,0,0,2694,0,350
CS001205000004,1988,100,128,286,346,368,760
CS001205000006,3337,635,60,198,2004,80,360
...,...,...,...,...,...,...,...
CS051513000004,551,318,0,0,233,0,0
CS051515000002,265,0,0,165,0,100,0
CS052212000002,192,0,0,0,102,90,0


**（松尾コメント）**\
一括で条件別で処理できるのは、データマイニング的に特徴量を作成する際に超便利

## Ver1.1（応用の修正）

In [154]:
%%time
# 顧客×カテゴリで集約して、横持に変換
_tmp = _df.groupby(["customer_id", "category_major_cd"])["amount"].agg([("sum_amt", "sum")]).unstack().fillna(0).astype('Int64')

# 列名を成形
_tmp.columns = ['_'.join([n,m]) for n, m in _tmp.columns]

CPU times: user 66.5 ms, sys: 307 µs, total: 66.8 ms
Wall time: 63.9 ms


In [156]:
_tmp

,sum_amt_04,sum_amt_05,sum_amt_06,sum_amt_07,sum_amt_08,sum_amt_09
customer_id,,,,,,
CS001113000004,0,0,0,1298,0,0
CS001114000005,0,40,0,486,100,0
CS001115000010,0,0,0,2694,0,350
CS001205000004,100,128,286,346,368,760
CS001205000006,635,60,198,2004,80,360
...,...,...,...,...,...,...
CS051513000004,318,0,0,233,0,0
CS051515000002,0,0,165,0,100,0
CS052212000002,0,0,0,102,90,0


---
> P-070: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過日数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。

In [98]:
%%time
# レシート情報の加工
_rec = df_receipt[
    ["customer_id", "sales_ymd"]
    
# 顧客単位で同日のデータを削除    
].copy().drop_duplicates()#.sort_values("customer_id")

# 日付型に変換
_rec["sales_ymd_dt"] = pd.to_datetime(_rec["sales_ymd"].astype('str'))
# _rec.head()

CPU times: user 83.2 ms, sys: 0 ns, total: 83.2 ms
Wall time: 79.9 ms


In [103]:
%%time
# 顧客データの加工
_cus = df_customer[["customer_id", "application_date"]].copy() #.sort_values("customer_id")

# 日付型に変換
_cus["app_date_dt"] = pd.to_datetime(_cus["application_date"])

# 各種確認
# print("・データ型:\n{}\n ・データ構造:{} \n ・ユニーク件数:{}".format(_cus.dtypes, _cus.shape, _cus["customer_id"].nunique()))

CPU times: user 13.4 ms, sys: 472 µs, total: 13.8 ms
Wall time: 11.7 ms


In [88]:
# データ結合
_df = pd.merge(
    _rec[["customer_id", "sales_ymd_dt"]], 
    _cus[["customer_id", "app_date_dt"]], 
    how="inner", 
    on="customer_id"
    
)

# 会員加入日からの経過日数
_df["n_days"] = _df["sales_ymd_dt"]-_df["app_date_dt"]
_df.head(10)

,customer_id,sales_ymd_dt,app_date_dt,n_days
0,CS006214000001,2018-11-03,2015-02-01,1371 days
1,CS006214000001,2017-05-09,2015-02-01,828 days
2,CS006214000001,2017-06-08,2015-02-01,858 days
3,CS006214000001,2018-10-28,2015-02-01,1365 days
4,CS006214000001,2019-09-08,2015-02-01,1680 days
5,CS006214000001,2018-01-31,2015-02-01,1095 days
6,CS006214000001,2017-07-05,2015-02-01,885 days
7,CS006214000001,2018-11-10,2015-02-01,1378 days
8,CS006214000001,2019-04-10,2015-02-01,1529 days
9,CS006214000001,2019-06-01,2015-02-01,1581 days


## Ver1.1
>P-070: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過日数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。

In [158]:
# 決済情報
_rec = df_receipt[["customer_id", "sales_ymd"]].drop_duplicates()
_rec["sales_ymd_dt"] = pd.to_datetime(_rec["sales_ymd"].astype('str'))

# 顧客情報
_cus = df_customer[["customer_id", "application_date"]].copy()
_cus["application_date_dt"] = pd.to_datetime(_cus["application_date"])


# 決済情報に会員申込日を結合
_df = pd.merge(_rec, _cus, on="customer_id", how="inner")

# 経過日数を計算
_df["n_days"] = _df["sales_ymd_dt"] - _df["application_date_dt"]

_df.head(10)

,customer_id,sales_ymd,sales_ymd_dt,application_date,application_date_dt,n_days
0,CS006214000001,20181103,2018-11-03,20150201,2015-02-01,1371 days
1,CS006214000001,20170509,2017-05-09,20150201,2015-02-01,828 days
2,CS006214000001,20170608,2017-06-08,20150201,2015-02-01,858 days
3,CS006214000001,20181028,2018-10-28,20150201,2015-02-01,1365 days
4,CS006214000001,20190908,2019-09-08,20150201,2015-02-01,1680 days
5,CS006214000001,20180131,2018-01-31,20150201,2015-02-01,1095 days
6,CS006214000001,20170705,2017-07-05,20150201,2015-02-01,885 days
7,CS006214000001,20181110,2018-11-10,20150201,2015-02-01,1378 days
8,CS006214000001,20190410,2019-04-10,20150201,2015-02-01,1529 days
9,CS006214000001,20190601,2019-06-01,20150201,2015-02-01,1581 days


**（松尾コメント）**\
※処理は同じ

---
> P-071: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過月数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1ヶ月未満は切り捨てること。

In [107]:
%%time
# レシートデータの加工
_rec = pd.concat(
    [
        df_receipt["customer_id"],
        
        # 日付型に変換
        pd.to_datetime(df_receipt["sales_ymd"].astype("str"))
        
    # 横結合
    ], axis=1
    
# 重複削除：同一顧客同日データを削除    
).drop_duplicates()

# _rec.head()
# 1行で書けるけど、重複がある分、日付変換に余計な処理時間がかかる

CPU times: user 161 ms, sys: 9.34 ms, total: 170 ms
Wall time: 166 ms


In [108]:
%%time 
# 顧客データの加工
_cus = pd.concat(
    [
        df_customer["customer_id"],
        
        # 日付型に変換
        pd.to_datetime(df_customer["application_date"])
        
    # 横結合   
    ], axis=1
    
)
# 1行で書けるし、早い

CPU times: user 9.29 ms, sys: 0 ns, total: 9.29 ms
Wall time: 8.48 ms


In [109]:
from datetime import datetime as dt

In [113]:
%%time 
# データ結合
_df = pd.merge(_rec, _cus, how="inner", on="customer_id").sort_values("customer_id")

# 経過月数を計算
_df["diff_month"] = (
    
    # 年数
    _df["sales_ymd"].dt.year - _df["application_date"].dt.year
    
)*12 + (
    
    # 月数
    _df["sales_ymd"].dt.month - _df["application_date"].dt.month
    
)

# 日にちは非考慮
_df.head(10)

CPU times: user 86.5 ms, sys: 402 µs, total: 86.9 ms
Wall time: 83 ms


,customer_id,sales_ymd,application_date,diff_month
29764,CS001113000004,2019-03-08,2015-11-05,40
9915,CS001114000005,2019-07-31,2016-04-12,39
9914,CS001114000005,2018-05-03,2016-04-12,25
14331,CS001115000010,2019-04-05,2015-04-17,48
14332,CS001115000010,2018-07-01,2015-04-17,39
14333,CS001115000010,2017-12-28,2015-04-17,32
3204,CS001205000004,2019-06-25,2016-06-15,36
3203,CS001205000004,2019-03-12,2016-06-15,33
3200,CS001205000004,2018-08-21,2016-06-15,26
3202,CS001205000004,2017-09-14,2016-06-15,15


## Ver1.1
>P-071: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過月数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1ヶ月未満は切り捨てること。

In [5]:
# 決済情報
_rec = df_receipt[["customer_id", "sales_ymd"]].drop_duplicates()
_rec["sales_ymd_dt"] = pd.to_datetime(_rec["sales_ymd"].astype('str'))

# 顧客情報
_cus = df_customer[["customer_id", "application_date"]].copy()
_cus["application_date_dt"] = pd.to_datetime(_cus["application_date"])


# 決済情報に会員申込日を結合
_df = pd.merge(_rec, _cus, on="customer_id", how="inner")

In [1]:
# 日付を考慮した経過月数
!pip install MonthDelta
from monthdelta import monthmod

  Created wheel for MonthDelta: filename=MonthDelta-0.9.1-py3-none-any.whl size=3110 sha256=8d0f071d51de28b8a598a658fb34cb3138066ef4efc4c130bf78911d8ff5b0c7
  Stored in directory: /home/jovyan/.cache/pip/wheels/43/47/38/1964a4a0eaf7b4e886ed712b5d9375438d7124837a3dae616a
Successfully built MonthDelta


In [22]:
%%time
_df["n_month"] = _df.apply(lambda x : monthmod(x["application_date_dt"], x["sales_ymd_dt"])[0].months, axis=1)

# _df.sort_values("n_month", ascending=True)
_df.sort_values("customer_id").head(10)

CPU times: user 1.65 s, sys: 0 ns, total: 1.65 s
Wall time: 1.65 s


,customer_id,sales_ymd,sales_ymd_dt,application_date,application_date_dt,n_month
29764,CS001113000004,20190308,2019-03-08,20151105,2015-11-05,40
9915,CS001114000005,20190731,2019-07-31,20160412,2016-04-12,39
9914,CS001114000005,20180503,2018-05-03,20160412,2016-04-12,24
14331,CS001115000010,20190405,2019-04-05,20150417,2015-04-17,47
14332,CS001115000010,20180701,2018-07-01,20150417,2015-04-17,38
14333,CS001115000010,20171228,2017-12-28,20150417,2015-04-17,32
3204,CS001205000004,20190625,2019-06-25,20160615,2016-06-15,36
3203,CS001205000004,20190312,2019-03-12,20160615,2016-06-15,32
3200,CS001205000004,20180821,2018-08-21,20160615,2016-06-15,26
3202,CS001205000004,20170914,2017-09-14,20160615,2016-06-15,14


---
> P-072: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過年数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い。（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1年未満は切り捨てること。

In [121]:
%%time
# レシートデータの加工
_rec = pd.concat(
    [
        df_receipt["customer_id"], 
        pd.to_datetime(df_receipt["sales_ymd"].astype('str'))
    ],axis=1
    
).drop_duplicates()#.sort_values("customer_id")

CPU times: user 168 ms, sys: 0 ns, total: 168 ms
Wall time: 163 ms


In [122]:
%%time
# 顧客データの加工
_cus = pd.concat(
    [
        df_customer["customer_id"],
        pd.to_datetime(df_customer["application_date"])
    ], axis=1
    
)

CPU times: user 10.2 ms, sys: 0 ns, total: 10.2 ms
Wall time: 8.91 ms


In [129]:
# データ結合
_df = pd.merge(_rec, _cus, how="inner", on="customer_id")

In [130]:
%%time
# 経過年数を計算
_df["diff_year"] = _df[["sales_ymd", "application_date"]].apply(lambda x: relativedelta(x[0], x[1]).years, axis=1)
_df.head(10)

CPU times: user 2.67 s, sys: 0 ns, total: 2.67 s
Wall time: 2.66 s


,customer_id,sales_ymd,application_date,diff_year
0,CS006214000001,2018-11-03,2015-02-01,3
1,CS006214000001,2017-05-09,2015-02-01,2
2,CS006214000001,2017-06-08,2015-02-01,2
3,CS006214000001,2018-10-28,2015-02-01,3
4,CS006214000001,2019-09-08,2015-02-01,4
5,CS006214000001,2018-01-31,2015-02-01,2
6,CS006214000001,2017-07-05,2015-02-01,2
7,CS006214000001,2018-11-10,2015-02-01,3
8,CS006214000001,2019-04-10,2015-02-01,4
9,CS006214000001,2019-06-01,2015-02-01,4


## Ver1.1
>P-072: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過年数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い。（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1年未満は切り捨てること。

In [28]:
%%time
# 決済情報
_rec = df_receipt[["customer_id", "sales_ymd"]].drop_duplicates()
_rec["sales_ymd_dt"] = pd.to_datetime(_rec["sales_ymd"].astype('str'))

# 顧客情報
_cus = df_customer[["customer_id", "application_date"]].copy()
_cus["app_date_dt"] = pd.to_datetime(_cus["application_date"])

# 決済情報に顧客情報を結合
_df = pd.merge(_rec, _cus, on="customer_id", how="inner")

# 経過年数
_df["n_year"] = _df.apply(lambda x : relativedelta(x["sales_ymd_dt"], x["app_date_dt"]).years, axis=1)

_df.sort_values("customer_id").head(10)

CPU times: user 2.93 s, sys: 0 ns, total: 2.93 s
Wall time: 2.92 s


,customer_id,sales_ymd,sales_ymd_dt,application_date,app_date_dt,n_year
29764,CS001113000004,20190308,2019-03-08,20151105,2015-11-05,3
9915,CS001114000005,20190731,2019-07-31,20160412,2016-04-12,3
9914,CS001114000005,20180503,2018-05-03,20160412,2016-04-12,2
14331,CS001115000010,20190405,2019-04-05,20150417,2015-04-17,3
14332,CS001115000010,20180701,2018-07-01,20150417,2015-04-17,3
14333,CS001115000010,20171228,2017-12-28,20150417,2015-04-17,2
3204,CS001205000004,20190625,2019-06-25,20160615,2016-06-15,3
3203,CS001205000004,20190312,2019-03-12,20160615,2016-06-15,2
3200,CS001205000004,20180821,2018-08-21,20160615,2016-06-15,2
3202,CS001205000004,20170914,2017-09-14,20160615,2016-06-15,1


---
> P-073: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からのエポック秒による経過時間を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。なお、時間情報は保有していないため各日付は0時0分0秒を表すものとする。

In [29]:
%%time
# レシート情報の加工
_rec = pd.concat(
    [
        df_receipt["customer_id"],
        
        #日付型に変換
        pd.to_datetime(df_receipt["sales_ymd"].astype('str'))
        
    ], axis=1

# 同一顧客で同日データで重複削除
).drop_duplicates()

CPU times: user 170 ms, sys: 387 µs, total: 170 ms
Wall time: 165 ms


In [30]:
%%time
# 顧客データの加工
_cus = pd.concat(
    [
        df_customer["customer_id"],
        
        # 日付型に変換
        pd.to_datetime(df_customer["application_date"])
        
    ], axis=1
    
)

CPU times: user 11.7 ms, sys: 47 µs, total: 11.8 ms
Wall time: 10.4 ms


## Ver1.1

In [31]:
%%time
# レシート情報
_rec = df_receipt[["customer_id", "sales_ymd"]].drop_duplicates()
_rec["sales_ymd"] = pd.to_datetime(_rec["sales_ymd"].astype('str'))

# 顧客情報
_cus = df_customer[["customer_id", "application_date"]].copy()
_cus["application_date"] = pd.to_datetime(_cus["application_date"])

CPU times: user 89.6 ms, sys: 10.1 ms, total: 99.7 ms
Wall time: 97 ms


In [32]:
# データ結合
_df = pd.merge(_rec, _cus, how="inner", on="customer_id")
_df.head()

,customer_id,sales_ymd,application_date
0,CS006214000001,2018-11-03,2015-02-01
1,CS006214000001,2017-05-09,2015-02-01
2,CS006214000001,2017-06-08,2015-02-01
3,CS006214000001,2018-10-28,2015-02-01
4,CS006214000001,2019-09-08,2015-02-01


In [172]:
# 経過時間（エポック）
# 10^9で割る理由がわからない
_df["diff_seconds"] = (_df["sales_ymd"] - _df["application_date"]).view('int64')/(10**9)
_df.head(10)

,customer_id,sales_ymd,application_date,diff_seconds
0,CS006214000001,2018-11-03,2015-02-01,118454400.0
1,CS006214000001,2017-05-09,2015-02-01,71539200.0
2,CS006214000001,2017-06-08,2015-02-01,74131200.0
3,CS006214000001,2018-10-28,2015-02-01,117936000.0
4,CS006214000001,2019-09-08,2015-02-01,145152000.0
5,CS006214000001,2018-01-31,2015-02-01,94608000.0
6,CS006214000001,2017-07-05,2015-02-01,76464000.0
7,CS006214000001,2018-11-10,2015-02-01,119059200.0
8,CS006214000001,2019-04-10,2015-02-01,132105600.0
9,CS006214000001,2019-06-01,2015-02-01,136598400.0


---
> P-074: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、当該週の月曜日からの経過日数を計算し、売上日、当該週の月曜日付とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値でデータを保持している点に注意）。

In [34]:
%%time
# レシート情報の加工
_rec = pd.concat(
    [
        df_receipt["customer_id"],
        pd.to_datetime(df_receipt["sales_ymd"].astype('str'))
    
    ], axis=1
    
).drop_duplicates()

CPU times: user 155 ms, sys: 10.2 ms, total: 165 ms
Wall time: 160 ms


## Ver1.1

In [33]:
%%time
# レシート情報
_rec = df_receipt[["customer_id", "sales_ymd"]].drop_duplicates()
_rec["sales_ymd"] = pd.to_datetime(_rec["sales_ymd"].astype('str'))

CPU times: user 80.1 ms, sys: 0 ns, total: 80.1 ms
Wall time: 77.7 ms


In [188]:
# 曜日（0:月, 1:火, ..., 6:日）を取得 = 月曜からの経過日数
_rec["date"] = _rec["sales_ymd"].apply(dt.weekday)
# 月曜日の日付を計算
_rec["monday"] =_rec[["sales_ymd", "date"]].apply(lambda x: x[0]-relativedelta(days=x[1]), axis=1) 
_rec.head(10)

,customer_id,sales_ymd,date,monday
0,CS006214000001,2018-11-03,5,2018-10-29
1,CS008415000097,2018-11-18,6,2018-11-12
2,CS028414000014,2017-07-12,2,2017-07-10
3,ZZ000000000000,2019-02-05,1,2019-02-04
4,CS025415000050,2018-08-21,1,2018-08-20
5,CS003515000195,2019-06-05,2,2019-06-03
6,CS024514000042,2018-12-05,2,2018-12-03
7,CS040415000178,2019-09-22,6,2019-09-16
8,ZZ000000000000,2017-05-04,3,2017-05-01
9,CS027514000015,2019-10-10,3,2019-10-07


---
> P-075: 顧客データフレーム（df_customer）からランダムに1%のデータを抽出し、先頭から10件データを抽出せよ。

---
> P-076: 顧客データフレーム（df_customer）から性別（gender_cd）の割合に基づきランダムに10%のデータを層化抽出し、性別ごとに件数を集計せよ。

---
> P-077: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を平均から3σ以上離れたものとする。結果は10件表示させれば良い。

In [211]:
_df = df_receipt[
    
    # 顧客ID, 売上金額
    ["customer_id", "amount"]

].query(
    
    # 非会員を除く
    'customer_id.str.contains("^[^Z]", regex=True)', engine="python"
    
).groupby("customer_id").agg(
 
    # 売上金額合計
    {"amount":"sum"}
    
).rename(
    
    columns={"amount":"sum_amt"}
    
).reset_index()

In [234]:
%%time
# 平均値を計算
_mean = np.mean(_df["sum_amt"])
# 標準偏差を計算
_std = np.std(_df["sum_amt"])

# 上限下限
_higt = _mean + 3*_std
_low  = _mean - 3*_std

# 確認用
# print("平均値:{:.1f}, 標準偏差:{:.1f}, 95信頼区間:[{:.1f}, {:.1f}]".format(_mean, _std, _low, _higt))

_df.query(
    'sum_amt <= @_low | sum_amt >= @_higt'
).head(10)
# _df.shape

CPU times: user 6.29 ms, sys: 370 µs, total: 6.66 ms
Wall time: 3.86 ms


,customer_id,sum_amt
332,CS001605000009,18925
1755,CS006415000147,12723
1817,CS006515000023,18372
1833,CS006515000125,12575
1841,CS006515000209,11373
1870,CS007115000006,11528
1941,CS007514000056,13293
1943,CS007514000094,15735
1951,CS007515000107,11188
1997,CS007615000026,11959


In [236]:
%%time
# N(0, 1)に標準化後、±3以上を抽出
_df[np.abs(preprocessing.scale(_df["sum_amt"])) >= 3].head(10)
# _df.shape

CPU times: user 0 ns, sys: 6.22 ms, total: 6.22 ms
Wall time: 3.43 ms


,customer_id,sum_amt
332,CS001605000009,18925
1755,CS006415000147,12723
1817,CS006515000023,18372
1833,CS006515000125,12575
1841,CS006515000209,11373
1870,CS007115000006,11528
1941,CS007514000056,13293
1943,CS007514000094,15735
1951,CS007515000107,11188
1997,CS007615000026,11959


## Ver1.1
>P-077: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を平均から3σ以上離れたものとする。結果は10件表示させれば良い。

In [40]:
# 顧客単位：売上金額合計
_rec = df_receipt[~df_receipt["customer_id"].str.startswith("Z")][["customer_id", "amount"]].groupby("customer_id")["amount"].agg([("sum_amt", 'sum')])

# 標準化：N(0,1)
_rec["z_scr"] = preprocessing.scale(_rec["sum_amt"])

# ±3シグマ以上
_rec[np.abs(_rec["z_scr"]) >= 3].head(10)

,sum_amt,z_scr
customer_id,,
CS001605000009,18925,6.019921
CS006415000147,12723,3.740202
CS006515000023,18372,5.816651
CS006515000125,12575,3.685800
CS006515000209,11373,3.243972
CS007115000006,11528,3.300946
CS007514000056,13293,3.949721
CS007514000094,15735,4.847347
CS007515000107,11188,3.175970


---
> P-078: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を第一四分位と第三四分位の差であるIQRを用いて、「第一四分位数-1.5×IQR」よりも下回るもの、または「第三四分位数+1.5×IQR」を超えるものとする。結果は10件表示させれば良い。

In [238]:
# 集約データの作成
_df = df_receipt[["customer_id", "amount"]].query(
    
    'customer_id.str.contains("^[^Z]", regex=True)', engine="python"
    
).groupby("customer_id").amount.sum().reset_index().rename(
    
    columns={"amount":"sum_amt"}
    
)

In [265]:
# 四分位を計算
# 0, 25, 50, 75, 100
_pct = pd.DataFrame(_df["sum_amt"].quantile([i/4 for i in range(5)]))

# 範囲の計算
_iqr = _pct.iloc[3, 0] - _pct.iloc[1, 0]

# 上下限の計算
_high = _pct.iloc[3, 0] + (1.5 * _iqr)
_low  = _pct.iloc[1, 0] - (1.5 * _iqr)

# 外れ値の抽出
_df.query('sum_amt < @_low | sum_amt > @_high').head(10)

,customer_id,sum_amt
98,CS001414000048,8584
332,CS001605000009,18925
549,CS002415000594,9568
1180,CS004414000181,9584
1558,CS005415000137,8734
1733,CS006414000001,9156
1736,CS006414000029,9179
1752,CS006415000105,10042
1755,CS006415000147,12723
1757,CS006415000157,10648


## Ver1.1

In [53]:
%%time
# 顧客単位に売上金額合計を計算
_rec = df_receipt[~df_receipt["customer_id"].str.startswith('Z')][["customer_id", "amount"]].groupby("customer_id")["amount"].agg([('sum_amt', 'sum')])

# 四分位点を計算
_qtl = _rec['sum_amt'].quantile([i/4 for i in range(4)]).to_frame()
# 四分位範囲
IQR = _qtl.loc[0.75, 'sum_amt'] - _qtl.loc[0.25, 'sum_amt']
# 上下限
HIGH = _qtl.loc[0.75, 'sum_amt'] + 1.5*IQR
LOW  = _qtl.loc[0.25, 'sum_amt'] - 1.5*IQR

# 外れ値10件表示
_rec[(_rec['sum_amt'] < LOW) | (_rec['sum_amt'] > HIGH)].head(10)

CPU times: user 79.9 ms, sys: 0 ns, total: 79.9 ms
Wall time: 77.9 ms


,sum_amt
customer_id,
CS001414000048,8584
CS001605000009,18925
CS002415000594,9568
CS004414000181,9584
CS005415000137,8734
CS006414000001,9156
CS006414000029,9179
CS006415000105,10042
CS006415000147,12723


---
> P-079: 商品データフレーム（df_product）の各項目に対し、欠損数を確認せよ。

---
> P-080: 商品データフレーム（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たなdf_product_1を作成せよ。なお、削除前後の件数を表示させ、前設問で確認した件数だけ減少していることも確認すること。

---
> P-081: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たなdf_product_2を作成せよ。なお、平均値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

In [277]:
round(df_product[["unit_price", "unit_cost"]].mean(skipna=True), 0)

unit_price    403.0
unit_cost     302.0
dtype: float64

In [279]:
df_product_2 = df_product.fillna(
    
    # 平均値で補完（欠損値除外）
    round(df_product[["unit_price", "unit_cost"]].mean(skipna=True), 0)
)
df_product_2.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             0
dtype: int64

## Ver1.1

In [65]:
df_product_2 = df_product.fillna(round(df_product[["unit_price", "unit_cost"]].mean(skipna=True), 0))
df_product_2.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             0
dtype: int64

---
> P-082: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの中央値で補完した新たなdf_product_3を作成せよ。なお、中央値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

In [2]:
df_product_3 = df_product.fillna(
    
    # 中央値で補完
    round(df_product[["unit_price", "unit_cost"]].median(skipna=True), 0)
    
)
df_product_3.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             0
dtype: int64

## Ver1.1

In [67]:
df_product_3 = df_product.fillna(round(df_product[['unit_price', 'unit_cost']].median(skipna=True), 0))
df_product_3.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             0
dtype: int64

---
> P-083: 単価（unit_price）と原価（unit_cost）の欠損値について、各商品の小区分（category_small_cd）ごとに算出した中央値で補完した新たなdf_product_4を作成せよ。なお、中央値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

In [3]:
# 欠損値確認
df_product.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            7
unit_cost             7
dtype: int64

In [94]:
%%time
# 小区分ごとに中央値を計算
_med =  df_product.groupby("category_small_cd").agg(
    
    {"unit_price":"median", "unit_cost":"median"}

).reset_index().rename(

    columns={"unit_price":"med_unit_price", "unit_cost":"med_unit_cost"}

# 小数点以下を四捨五入
).round()

# _med.isnull().sum()

CPU times: user 12.2 ms, sys: 0 ns, total: 12.2 ms
Wall time: 8.76 ms


In [22]:
%%time
# 小区分ごとの中央値を結合
_df = pd.merge(
    # 商品データ
    df_product,
    # 小区分ごとの中央値
    _med,
    # 左外部結合
    how="left",
    # 小区分
    on="category_small_cd"
)

CPU times: user 11.4 ms, sys: 0 ns, total: 11.4 ms
Wall time: 8.32 ms


In [26]:
# 欠損値補完
# 単価
_df["unit_price"] = _df["unit_price"].mask(_df["unit_price"].isnull(), _df["med_unit_price"])

# 原価
_df["unit_cost"] = _df["unit_cost"].mask(_df["unit_cost"].isnull(), _df["med_unit_cost"])


# 補完結果の確認
# _df[df_product["unit_price"].isnull()]
# _df[df_product["unit_cost"].isnull()]
_df.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             0
med_unit_price        0
med_unit_cost         0
dtype: int64

## Ver1.1
>P-083: 単価（unit_price）と原価（unit_cost）の欠損値について、各商品の小区分（category_small_cd）ごとに算出した中央値で補完した新たなdf_product_4を作成せよ。なお、中央値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

In [95]:
%%time
# 集約関数定義
def def_grp(x):
    d={}
    d['med_price'] = round(x['unit_price'].median(skipna=True), 0)
    d['med_cost']  = round(x['unit_cost'].median(skipna=True), 0)
    return pd.Series(d)

_cat = df_product.groupby('category_small_cd').apply(def_grp).reset_index()

CPU times: user 127 ms, sys: 21.1 ms, total: 148 ms
Wall time: 129 ms


In [97]:
# 小区分で結合
_df = df_product.merge(_cat, on='category_small_cd', how='inner')
# 欠損値補完
_df['unit_price'].mask(_df['unit_price'].isnull(), _df['med_price'], inplace=True)
_df['unit_cost'].mask(_df['unit_cost'].isnull(), _df['med_cost'], inplace=True)

In [93]:
# _df[df_product["unit_price"].isnull()]
# _df[df_product["unit_cost"].isnull()]
_df.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             0
med_price             0
med_cost              0
dtype: int64

**（松尾コメント）**\
groupbyの集約関数の処理は、agg辞書渡し⇒apply集約関数定義\
※実行時間は増加するが、前者が廃止予定なので慣れておく

---
> P-084: 顧客データフレーム（df_customer）の全顧客に対し、全期間の売上金額に占める2019年売上金額の割合を計算せよ。ただし、販売実績のない場合は0として扱うこと。そして計算した割合が0超のものを抽出せよ。 結果は10件表示させれば良い。また、作成したデータにNAやNANが存在しないことを確認せよ。

In [43]:
#
#レシート情報の加工
# ・顧客単位に集約
#
_rec1 = pd.concat(
    [
        df_receipt[["customer_id", "amount"]],
        
        # 売上年（yyyy）を抽出
        pd.to_datetime(df_receipt["sales_ymd"].astype('str')).dt.year
        
    ], axis=1    
)

# 顧客単位に、全期間売上金額合計と2019年の売上金額合計を計算
_rec2 = pd.merge(
    
    # 全期間：売上金額合計
    _rec1.groupby("customer_id").amount.sum().reset_index().rename(columns={"amount":"sum_amt_all"}),
    
    # 2019年：売上金額合計
    _rec1.query('sales_ymd == 2019').groupby("customer_id").amount.sum().reset_index().rename(columns={"amount":"sum_amt_2019"}),
    
    # 顧客ID
    on="customer_id",
    # 左外部結合（2019年の売り上げがない先は欠損値
    how="left"
    
# 欠損値の0補完
).fillna(0)

# 全期間に対する2019年割合を計算
_rec2["rat_amt_2019"] = np.divide(_rec2["sum_amt_2019"], _rec2["sum_amt_all"])
_rec2.isnull().sum()

customer_id     0
sum_amt_all     0
sum_amt_2019    0
rat_amt_2019    0
dtype: int64

In [52]:
# データ結合
_df = pd.merge(
    
    # 全顧客情報
    df_customer["customer_id"],
    
    # レシート情報
    _rec2,
    
    # 顧客ID
    on="customer_id",
    # 左外部結合
    how="left"
    
# 欠損値補完 = 売上情報がない先は0（円, %）補完
).fillna(0).sort_values("customer_id")

print(">>欠損値確認\n{}".format(_df.isnull().sum()))
_df.query('rat_amt_2019 > 0').head(10)

>>欠損値確認
customer_id     0
sum_amt_all     0
sum_amt_2019    0
rat_amt_2019    0
dtype: int64


,customer_id,sum_amt_all,sum_amt_2019,rat_amt_2019
3718,CS001113000004,1298.0,1298.0,1.000000
19087,CS001114000005,626.0,188.0,0.300319
16778,CS001115000010,3044.0,578.0,0.189882
2786,CS001205000004,1988.0,702.0,0.353119
8428,CS001205000006,3337.0,486.0,0.145640
16733,CS001211000025,456.0,456.0,1.000000
19987,CS001212000070,456.0,456.0,1.000000
7728,CS001214000009,4685.0,664.0,0.141729
8924,CS001214000017,4132.0,2962.0,0.716844
18827,CS001214000048,2374.0,1889.0,0.795703


## Ver1.1
>P-084: 顧客データフレーム（df_customer）の全顧客に対し、全期間の売上金額に占める2019年売上金額の割合を計算せよ。ただし、販売実績のない場合は0として扱うこと。そして計算した割合が0超のものを抽出せよ。 結果は10件表示させれば良い。また、作成したデータにNAやNANが存在しないことを確認せよ。

⇒2019年の購入実績がある人

In [107]:
YYYY = 2019

In [109]:
# 購入年を計算
_rec = df_receipt[['customer_id', 'amount', 'sales_ymd']].copy()
_rec['sales_yyyy'] = pd.to_datetime(_rec['sales_ymd'].astype('str')).dt.year

# 2019年に購入実績がある人
_trg = _rec['customer_id'][_rec['sales_yyyy'] == YYYY].drop_duplicates()  

In [115]:
# 売上金額合計を計算
_df = _rec[_rec['customer_id'].isin(_trg)][['customer_id', 'amount', 'sales_yyyy']].groupby('customer_id').apply(
    
    lambda x: pd.Series(
        [
            # 全期間合計
            x['amount'].sum(),
            # 2019年合計
            x[x['sales_yyyy'] == YYYY]['amount'].sum()
            
        ], index=['sum_amt_all', 'sum_amt_%s' %  YYYY]
        
    )
    
)

# 全体割合を計算
_df['rat_amt_%s' % YYYY] = np.divide(_df['sum_amt_%s' % YYYY], _df['sum_amt_all'])
_df.head(10)

,sum_amt_all,sum_amt_2019,rat_amt_2019
customer_id,,,
CS001113000004,1298,1298,1.000000
CS001114000005,626,188,0.300319
CS001115000010,3044,578,0.189882
CS001205000004,1988,702,0.353119
CS001205000006,3337,486,0.145640
CS001211000025,456,456,1.000000
CS001212000070,456,456,1.000000
CS001214000009,4685,664,0.141729
CS001214000017,4132,2962,0.716844


In [120]:
# 結果の確認
_rec[_rec['customer_id'].isin(_df.index[:2])].sort_values('customer_id')

,customer_id,amount,sales_ymd,sales_yyyy
31349,CS001113000004,98,20190308,2019
35811,CS001113000004,1200,20190308,2019
2818,CS001114000005,100,20180503,2018
7131,CS001114000005,338,20180503,2018
46293,CS001114000005,40,20190731,2019
91976,CS001114000005,148,20190731,2019


**（松尾コメント）**\
・年度の指定を一元化\
・必要なデータのみ処理（問題文の手順とは異なるが本質は処理している）\
・isinを使用し、結果の確認

---
> P-085: 顧客データフレーム（df_customer）の全顧客に対し、郵便番号（postal_cd）を用いて経度緯度変換用データフレーム（df_geocode）を紐付け、新たなdf_customer_1を作成せよ。ただし、複数紐づく場合は経度（longitude）、緯度（latitude）それぞれ平均を算出すること。


In [64]:
# 経度緯度変換用データの加工
_geo = df_geocode[
    # 郵便番号, 経度, 緯度
    ["postal_cd", "longitude", "latitude"]
    
# 同じ郵便番号の場合は平均値を計算
].groupby("postal_cd").agg(
    
    {"longitude":"mean", "latitude":"mean"}
    
).reset_index()

In [73]:
df_customer_1 = pd.merge(
    
    # 全顧客
    df_customer[["customer_id", "postal_cd", "address", "application_store_cd"]],
    # 経度緯度データ
    _geo,
    # 郵便番号
    on="postal_cd",
    # 左外部結合
    how="left"
    
)
df_customer_1.isnull().sum()

customer_id             0
postal_cd               0
address                 0
application_store_cd    0
longitude               0
latitude                0
dtype: int64

## Ver1.1

In [2]:
%%time
# 集約関数定義
def def_grp(x):
    d = {}
    d['longitude'] = x['longitude'].mean()
    d['latitude'] = x['latitude'].mean()
    return pd.Series(d)

_geo = df_geocode[['postal_cd', 'longitude', 'latitude']].groupby('postal_cd').apply(def_grp).reset_index()

CPU times: user 1min 1s, sys: 3.01 s, total: 1min 4s
Wall time: 1min


In [3]:
# 顧客情報に結合
df_customer_1 = pd.merge(df_customer, _geo, on='postal_cd', how='inner')

# 欠損数の確認
df_customer_1.isnull().sum()

customer_id             0
customer_name           0
gender_cd               0
gender                  0
birth_day               0
age                     0
postal_cd               0
address                 0
application_store_cd    0
application_date        0
status_cd               0
longitude               0
latitude                0
dtype: int64

---
> P-086: 前設問で作成した緯度経度つき顧客データフレーム（df_customer_1）に対し、申込み店舗コード（application_store_cd）をキーに店舗データフレーム（df_store）と結合せよ。そして申込み店舗の緯度（latitude）・経度情報（longitude)と顧客の緯度・経度を用いて距離（km）を求め、顧客ID（customer_id）、顧客住所（address）、店舗住所（address）とともに表示せよ。計算式は簡易式で良いものとするが、その他精度の高い方式を利用したライブラリを利用してもかまわない。結果は10件表示すれば良い。

$$
緯度（ラジアン）：\phi \\
経度（ラジアン）：\lambda \\
距離L = 6371 * arccos(sin \phi_1 * sin \phi_2
+ cos \phi_1 * cos \phi_2 * cos(\lambda_1 − \lambda_2))
$$

In [76]:
# 重複確認
# df_store[["store_cd", "address", "longitude", "latitude"]].duplicated().sum()

0

In [110]:
# データ結合
_df = pd.merge(
    
    # 顧客情報
    df_customer_1.rename(
        # 変数名が重複するので変更
        columns={
            "address":"cus_address",
            "longitude":"cus_longitude",
            "latitude":"cus_latitude",
            
            # 結合用にRENAME
            "application_store_cd":"store_cd"
        }
    ),
    
    # 店舗情報
    df_store[
        ["store_cd", "address", "longitude", "latitude"]
    ].rename(
        # 同様の理由で変更
        columns={
            "address":"str_address",
            "longitude":"str_longitude",
            "latitude":"str_latitude"
        }
    ),
    
    # 店舗コード
    on="store_cd",
    #
    how="left"
    
).sort_values("customer_id")
# _df.isnull().sum() ⇒全店舗結合する

In [111]:
# 顧客住所と申込店舗住所との距離（km）を計算
# ! pip install geopy
from geopy.distance import geodesic

In [112]:
_df["distance"] = _df[
    # 顧客住所, 店舗住所（緯度経度）
    ["cus_latitude", "cus_longitude", "str_latitude", "str_longitude"]

].apply(
    lambda x:
    # 経度緯度から距離（km）を計算
    geodesic(x[0:2], x[2:4]).km, axis=1
)
_df[["customer_id", "cus_address", "str_address", "distance"]].head(10)

,customer_id,cus_address,str_address,distance
21641,CS001105000001,東京都大田区西六郷**********,東京都大田区仲六郷二丁目,1.479424
14158,CS001112000009,東京都大田区西馬込**********,東京都大田区仲六郷二丁目,4.012468
8864,CS001112000019,東京都大田区昭和島**********,東京都大田区仲六郷二丁目,3.785633
8073,CS001112000021,東京都大田区西六郷**********,東京都大田区仲六郷二丁目,1.479424
9837,CS001112000023,東京都大田区昭和島**********,東京都大田区仲六郷二丁目,3.785633
20438,CS001112000024,東京都大田区西六郷**********,東京都大田区仲六郷二丁目,1.479424
4520,CS001112000029,東京都大田区西六郷**********,東京都大田区仲六郷二丁目,1.479424
12531,CS001112000030,東京都大田区西六郷**********,東京都大田区仲六郷二丁目,1.479424
3718,CS001113000004,東京都大田区西六郷**********,東京都大田区仲六郷二丁目,1.479424
4463,CS001113000010,東京都大田区西六郷**********,東京都大田区仲六郷二丁目,1.479424


## Ver1.1
>P-086: 前設問で作成した緯度経度つき顧客データフレーム（df_customer_1）に対し、申込み店舗コード（application_store_cd）をキーに店舗データフレーム（df_store）と結合せよ。そして申込み店舗の緯度（latitude）・経度情報（longitude)と顧客の緯度・経度を用いて距離（km）を求め、顧客ID（customer_id）、顧客住所（address）、店舗住所（address）とともに表示せよ。計算式は簡易式で良いものとするが、その他精度の高い方式を利用したライブラリを利用してもかまわない。結果は10件表示すれば良い。

In [13]:
# 加盟店コードの重複確認
df_store['store_cd'].duplicated().sum()

0

In [29]:
# 顧客情報に加盟店情報を結合
_df = pd.merge(
    
    # 顧客情報の抽出
    df_customer_1[
        # 顧客ID, 顧客住所, 加盟店コード
        ['customer_id', 'postal_cd', 'address', 'longitude', 'latitude', 'application_store_cd']
    ].rename(
        # 結合キーの名称変更
        columns={'application_store_cd':'store_cd'}
    ),
    
    # 加盟店情報
    df_store[['store_cd', 'address', 'longitude', 'latitude']].rename(
        
        # 変数名変更
        columns={'address':'str_address', 'longitude':'str_longitude', 'latitude':'str_latitude'}
    ),
    
    # 加盟店コード
    on='store_cd',
    
    # 内部結合
    how='inner'
    
# 結果を合わせるために
).sort_values('customer_id')

In [30]:
# 顧客住所と申込店舗住所との距離（km）を計算
# ! pip install geopy
from geopy.distance import geodesic

In [32]:
_df['dist_km'] = _df.apply(lambda x : geodesic(x[['latitude', 'longitude']], x[['str_latitude', 'str_longitude']]).km, axis=1)

_df.head(10)

,customer_id,postal_cd,address,longitude,latitude,store_cd,str_address,str_longitude,str_latitude,distance,dist_km
5464,CS001105000001,144-0056,東京都大田区西六郷**********,139.70238,35.54137,S13001,東京都大田区仲六郷二丁目,139.7132,35.55135,1.479424,1.479424
5624,CS001112000009,143-0026,東京都大田区西馬込**********,139.70386,35.58670,S13001,東京都大田区仲六郷二丁目,139.7132,35.55135,4.012468,4.012468
5801,CS001112000019,143-0004,東京都大田区昭和島**********,139.74687,35.57153,S13001,東京都大田区仲六郷二丁目,139.7132,35.55135,3.785633,3.785633
5363,CS001112000021,144-0056,東京都大田区西六郷**********,139.70238,35.54137,S13001,東京都大田区仲六郷二丁目,139.7132,35.55135,1.479424,1.479424
5802,CS001112000023,143-0004,東京都大田区昭和島**********,139.74687,35.57153,S13001,東京都大田区仲六郷二丁目,139.7132,35.55135,3.785633,3.785633
5453,CS001112000024,144-0056,東京都大田区西六郷**********,139.70238,35.54137,S13001,東京都大田区仲六郷二丁目,139.7132,35.55135,1.479424,1.479424
5332,CS001112000029,144-0056,東京都大田区西六郷**********,139.70238,35.54137,S13001,東京都大田区仲六郷二丁目,139.7132,35.55135,1.479424,1.479424
5397,CS001112000030,144-0056,東京都大田区西六郷**********,139.70238,35.54137,S13001,東京都大田区仲六郷二丁目,139.7132,35.55135,1.479424,1.479424
5321,CS001113000004,144-0056,東京都大田区西六郷**********,139.70238,35.54137,S13001,東京都大田区仲六郷二丁目,139.7132,35.55135,1.479424,1.479424
5331,CS001113000010,144-0056,東京都大田区西六郷**********,139.70238,35.54137,S13001,東京都大田区仲六郷二丁目,139.7132,35.55135,1.479424,1.479424


---
> P-087:  顧客データフレーム（df_customer）では、異なる店舗での申込みなどにより同一顧客が複数登録されている。名前（customer_name）と郵便番号（postal_cd）が同じ顧客は同一顧客とみなし、1顧客1レコードとなるように名寄せした名寄顧客データフレーム（df_customer_u）を作成せよ。ただし、同一顧客に対しては売上金額合計が最も高いものを残すものとし、売上金額合計が同一もしくは売上実績の無い顧客については顧客ID（customer_id）の番号が小さいものを残すこととする。

In [119]:
# 顧客単位に売上金額の合計を計算
_rec = df_receipt[["customer_id", "amount"]].groupby("customer_id").amount.sum().reset_index().rename(columns={"amount":"sum_amt"})

In [5]:
# df_receipt[["customer_id", "amount"]].groupby("customer_id").agg({"amount":"sum"}).reset_index().rename(columns={"amount":"sum_amt"})

In [140]:
# 顧客情報に売上データを結合
_df = pd.merge(
    # 顧客データ
    df_customer[
        # 顧客ID, 顧客名, 郵便番号, 
        ["customer_id", "customer_name", "postal_cd"]
    ],
    
    # 売上データ
    _rec,
    
    # 顧客ID
    on="customer_id",
    # 左外部結合
    how="left"
   
).fillna(
    # 売上実績のない先は0円補完
    {"sum_amt":0}

# 名前, 郵便番号, 金額（高いほう）, ID（小さいほう）
).sort_values(
    
    ["customer_name", "postal_cd", "sum_amt", "customer_id"],
    
    ascending=[True, True, False, True]
    
)
# _df.isnull().sum()

In [141]:
# 重複個所の確認用
# _df["dup"] = _df.duplicated(["customer_name", "postal_cd"])
# _df.to_csv('chk.csv')

# 重複削除（先頭を残す）
df_customer_u = _df.drop_duplicates(
    
    # 顧客名と郵便番号で重複削除
    ["customer_name", "postal_cd"], keep='first'

)

print("削除前:{}, 削除後:{}".format(len(df_customer), len(df_customer_u)))

削除前:21971, 削除後:21941


## Ver1.1
>P-087: 顧客データフレーム（df_customer）では、異なる店舗での申込みなどにより同一顧客が複数登録されている。名前（customer_name）と郵便番号（postal_cd）が同じ顧客は同一顧客とみなし、1顧客1レコードとなるように名寄せした名寄顧客データフレーム（df_customer_u）を作成せよ。ただし、同一顧客に対しては売上金額合計が最も高いものを残すものとし、売上金額合計が同一もしくは売上実績の無い顧客については顧客ID（customer_id）の番号が小さいものを残すこととする。

In [40]:
# 顧客データに売上金額合計を結合
df_customer_u = df_customer.merge(
    
    # 売上金額合計
    df_receipt[['customer_id', 'amount']].groupby('customer_id')['amount'].agg([('sum_amt', 'sum')]), 
    on='customer_id', 
    how='left'

# 決済なし先を0円補完
).fillna({'sum_amt':0}).sort_values(
    
    # 顧客名, 住所（昇順）、合計金額（降順）、顧客ＩＤ（昇順）
    ['customer_name', 'postal_cd', 'sum_amt', 'customer_id'], ascending=[True, True, False, True]
    
# 顧客名と住所で重複削除
# ※ルール：合計金額高いほう、顧客ＩＤ小さいほう
).drop_duplicates(['customer_name', 'postal_cd'], keep='first')

In [41]:
print("削除前:{}, 削除後:{}".format(len(df_customer), len(df_customer_u)))

削除前:21971, 削除後:21941


---
> P-088: 前設問で作成したデータを元に、顧客データフレームに統合名寄IDを付与したデータフレーム（df_customer_n）を作成せよ。ただし、統合名寄IDは以下の仕様で付与するものとする。
>
> - 重複していない顧客：顧客ID（customer_id）を設定
> - 重複している顧客：前設問で抽出したレコードの顧客IDを設定

In [154]:
df_customer_n = pd.merge(
    
    # 顧客データ全体
    df_customer,
    
    # 名寄せ顧客データ
    df_customer_u[["customer_name","postal_cd", "customer_id"]].reset_index().rename(
        
        # 統合名寄ID
        columns={"customer_id":"customer_id_nys"}
        
    ),
    # 顧客名と郵便番号
    left_on=["customer_name","postal_cd"],
    right_on=["customer_name","postal_cd"],
    
    # 左外部結合
    how="left"
    
).drop("index", axis=1).sort_values(["customer_name","postal_cd"])
# _df.isnull().sum()

In [153]:
# 名寄せ確認用
# df_customer_n["dup"] = df_customer_n.duplicated(["customer_name","postal_cd"])
# df_customer_n.to_csv('chk.csv')

## Ver1.1

In [43]:
# 結合キー
key = ['customer_name', 'postal_cd']

In [46]:
# 顧客IDを名寄せ
df_customer_n = df_customer.merge(df_customer_u[key + ['customer_id']].rename(columns={'customer_id':'customer_id_nys'}), on=key, how='left')

In [63]:
# 確認用
df_customer_n[df_customer_n['customer_id_nys'].isin(df_customer_n[df_customer_n.duplicated(key)]['customer_id_nys'].drop_duplicates())].sort_values(key).head()

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,customer_id_nys
2013,CS016712000025,久野 みゆき,1,女性,1960-08-22,58,144-0052,東京都大田区蒲田**********,S13016,20150627,0-00000000-0,CS001515000422
15632,CS001515000422,久野 みゆき,1,女性,1960-08-22,58,144-0052,東京都大田区蒲田**********,S13001,20160703,7-20100517-7,CS001515000422
13423,CS038214000037,今 充則,0,男性,1949-04-09,69,246-0001,神奈川県横浜市瀬谷区卸本町**********,S13038,20180514,0-00000000-0,CS038214000037
21698,CS040601000007,今 充則,0,男性,1949-04-09,69,246-0001,神奈川県横浜市瀬谷区卸本町**********,S14040,20150308,0-00000000-0,CS038214000037
3731,CS001515000561,伴 芽以,1,女性,1959-06-24,59,144-0051,東京都大田区西蒲田**********,S13001,20170317,A-20091210-8,CS001515000561


**（松尾コメント）**\
結合キーが複数になる場合は事前に定義しておくと楽\

---
> P-閑話: df_customer_1, df_customer_nは使わないので削除する。

In [64]:
del df_customer_1, df_customer_n

---
> P-089: 売上実績のある顧客に対し、予測モデル構築のため学習用データとテスト用データに分割したい。それぞれ8:2の割合でランダムにデータを分割せよ。

In [65]:
%%time
_df = pd.merge(
    
    # 顧客情報
    df_customer,
    # レシート情報（売上実績あり先）
    df_receipt.drop_duplicates("customer_id"),
    # 顧客ID
    on="customer_id",
    # 内部結合
    how="inner"
    
)
df_train, df_test = train_test_split(_df, test_size=0.2, random_state=123)

print("訓練データ:", len(df_train)/len(_df))
print("テストデータ:", len(df_test)/len(_df))

訓練データ: 0.7999036840837949
テストデータ: 0.20009631591620516
CPU times: user 66.2 ms, sys: 0 ns, total: 66.2 ms
Wall time: 62.6 ms


## Ver1.1

In [66]:
%%time
df_train, df_est = train_test_split(
    
    # 買い物実績のある顧客
    df_customer[df_customer['customer_id'].isin(df_receipt['customer_id'].drop_duplicates())], test_size=0.2, random_state=123

)

print("訓練データ:", len(df_train)/len(_df))
print("テストデータ:", len(df_test)/len(_df))

訓練データ: 0.7999036840837949
テストデータ: 0.20009631591620516
CPU times: user 30.4 ms, sys: 0 ns, total: 30.4 ms
Wall time: 27 ms


**（松尾コメント）**\
結合しなくても、isinを使用すれば対象の顧客を抽出できる\
※処理速度も速い

---
> P-090: レシート明細データフレーム（df_receipt）は2017年1月1日〜2019年10月31日までのデータを有している。売上金額（amount）を月次で集計し、学習用に12ヶ月、テスト用に6ヶ月のモデル構築用データを3セット作成せよ。

In [161]:
df_receipt.dtypes

sales_ymd          int64
sales_epoch        int64
store_cd          object
receipt_no         int64
receipt_sub_no     int64
customer_id       object
product_cd        object
quantity           int64
amount             int64
dtype: object

In [79]:
%%time
# 売上年月を抽出
_rec = pd.concat(
    
    [
        # 顧客ID, 売上金額
        df_receipt[["customer_id", "amount"]],
        
        # 売上年月
        pd.to_datetime(df_receipt["sales_ymd"].astype('str')).dt.strftime('%Y%m').reset_index().rename(columns={"sales_ymd":"sales_ym"})
        
    ], axis=1
 
# 売上年月で集約
).groupby("sales_ym").amount.sum().reset_index()

# (12+6)*3 = 54ヶ月必要では？
_rec.shape

CPU times: user 847 ms, sys: 0 ns, total: 847 ms
Wall time: 841 ms


(34, 2)

## Ver1.1

In [107]:
%%time
# 年月情報を抽出
_rec = df_receipt[['sales_ymd', 'amount']].copy()
_rec['sales_ym'] = pd.to_datetime(_rec['sales_ymd'].astype('str')).dt.strftime('%Y%m')

# 年月で集約
_df = _rec.groupby('sales_ym')['amount'].agg([('sum_amt', 'sum')])

CPU times: user 818 ms, sys: 0 ns, total: 818 ms
Wall time: 815 ms


分割ルールがないので設定\
・それ以外の抽出する基準時点は、(全期間-訓練テスト期間)/(n-1)ヶ月ずらし\
※nは分割数、小数点切り捨て（=終点データは使用しないことあり）

In [159]:
MON_TRN = 12
MON_TST = 6
N = 6

# ずらす年月
N_MON = np.floor((len(_df)-(MON_TRN+MON_TST))/(N-1)).astype('int64')
_tmp = _df.copy()

In [160]:
for i in range(N):
    
    train = _df[i*N_MON : i*N_MON+MON_TRN].copy()
    train['flg%s' % i] = 0
    
    test = _df[i*N_MON+MON_TRN : i*N_MON+MON_TRN+MON_TST].copy()
    test['flg%s' % i] = 1 
    
    _tmp =  pd.concat([_tmp, pd.concat([train, test], axis=0)['flg%s' % i]], axis=1).astype('Int64')
    
_tmp

,sum_amt,flg0,flg1,flg2,flg3,flg4,flg5
sales_ym,,,,,,,
201701,902056,0,<NA>,<NA>,<NA>,<NA>,<NA>
201702,764413,0,<NA>,<NA>,<NA>,<NA>,<NA>
201703,962945,0,<NA>,<NA>,<NA>,<NA>,<NA>
201704,847566,0,0,<NA>,<NA>,<NA>,<NA>
201705,884010,0,0,<NA>,<NA>,<NA>,<NA>
201706,894242,0,0,<NA>,<NA>,<NA>,<NA>
201707,959205,0,0,0,<NA>,<NA>,<NA>
201708,954836,0,0,0,<NA>,<NA>,<NA>
201709,902037,0,0,0,<NA>,<NA>,<NA>


---
> P-091: 顧客データフレーム（df_customer）の各顧客に対し、売上実績のある顧客数と売上実績のない顧客数が1:1となるようにアンダーサンプリングで抽出せよ。

In [177]:
%%time
_df = pd.merge(
    
    # 顧客情報
    df_customer,
    
    # 売上実績データ
    pd.DataFrame(df_receipt["customer_id"].drop_duplicates()).rename(columns={"customer_id":"_id"}),
    
    # 顧客ID
    left_on="customer_id",
    right_on="_id",
    # 左外部結合
    how="left"
    
)

# 売上実績の有無フラグ
_df["flg_rec"] = (1 - _df["_id"].isnull())
_df.drop("_id", axis=1, inplace=True)
_df.head()

CPU times: user 65.7 ms, sys: 131 µs, total: 65.8 ms
Wall time: 62.9 ms


,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,flg_rec
0,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0,0
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0,0
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C,1
3,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0,0
4,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2,1


In [178]:
%%time
# フラグの値で1:1にサンプリング
def smpling(indt, flg):
    
    # データを分割
    _df1 = indt.query(flg+'==1')
    _df0 = indt.query(flg+'==0')
    
    # データ数を計算
    _len1 = len(_df1)
    _len0 = len(_df0)
    
    # 件数が少ないほうに合わせる
    if _len1 >= _len0:
        
        _df1 = _df1.sample(n=_len0, random_state=123)
        
    else:
        
        _df0 = _df0.sample(n=_len1, random_state=123)
        
    return _df1, _df0
    
_df1, _df0 = smpling(indt=_df, flg='flg_rec')

print(_df.groupby("flg_rec").customer_id.count(), _df1.shape, _df0.shape)

flg_rec
0    13665
1     8306
Name: customer_id, dtype: int64 (8306, 12) (8306, 12)
CPU times: user 27.9 ms, sys: 239 µs, total: 28.1 ms
Wall time: 23.3 ms


## Ver1.1
>P-091: 顧客データフレーム（df_customer）の各顧客に対し、売上実績のある顧客数と売上実績のない顧客数が1:1となるようにアンダーサンプリングで抽出せよ。

In [179]:
%%time
# indicatorの定義
dict = {'both':1, 'left_only':0}

# 顧客情報に売上実績有無を結合
_df = df_customer.merge(df_receipt['customer_id'].drop_duplicates(), on='customer_id', how='left', indicator=True).replace({'_merge':dict})

CPU times: user 97.5 ms, sys: 162 µs, total: 97.7 ms
Wall time: 93 ms


In [176]:
# 件数の確認
_df.groupby('_merge')['customer_id'].count()

_merge
0    13665
1     8306
Name: customer_id, dtype: int64

In [181]:
%%time
# アンダーサンプリング
from imblearn.under_sampling import RandomUnderSampler

# サンプリング
_smpl, _ = RandomUnderSampler(random_state=123).fit_resample(_df, _df._merge)

_smpl.groupby('_merge')['customer_id'].count()

CPU times: user 40.9 ms, sys: 19.9 ms, total: 60.8 ms
Wall time: 58.3 ms


_merge
0    8306
1    8306
Name: customer_id, dtype: int64

**（松尾コメント）**\
・結合有無はindicatorとdict={'both':1, 'left_only':0}, replace({'_merge':dict})が1行で書けて楽かも

---
> P-092: 顧客データフレーム（df_customer）では、性別に関する情報が非正規化の状態で保持されている。これを第三正規化せよ。

---
> P-093: 商品データフレーム（df_product）では各カテゴリのコード値だけを保有し、カテゴリ名は保有していない。カテゴリデータフレーム（df_category）と組み合わせて非正規化し、カテゴリ名を保有した新たな商品データフレームを作成せよ。

In [204]:
_pro = pd.merge(
    
    # 商品データ
    df_product,
    # カテゴリデータ
    df_category,
    
    # 全カテゴリ
    on=["category_major_cd", "category_medium_cd", "category_small_cd"],
    # 左外部結合
    how="left"
)
         
# カテゴリ名はすべて結合済み
_pro.isnull().sum()     

product_cd              0
category_major_cd       0
category_medium_cd      0
category_small_cd       0
unit_price              7
unit_cost               7
category_major_name     0
category_medium_name    0
category_small_name     0
dtype: int64

## Ver1.1

In [186]:
# 商品データにカテゴリ名を結合
# 結合キー
key = ['category_small_cd']
_df = df_product.merge(df_category[key + ['category_major_name', 'category_medium_name', 'category_small_name']], on=key, how='inner')
_df.isnull().sum()

product_cd              0
category_major_cd       0
category_medium_cd      0
category_small_cd       0
unit_price              7
unit_cost               7
category_major_name     0
category_medium_name    0
category_small_name     0
dtype: int64

---
> P-094: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [215]:
_pro.to_csv('./data/P094_df_product_category.csv', index=False, header=True, encoding='UTF-8')

## Ver1.1
> パスは定義しておく（SASでいうlibname）\
> 接頭辞は「ver110」

In [192]:
import pathlib as Path
data_path = Path.Path("./data")

prefix = 'ver110'

In [202]:
%%time
_df.to_csv(data_path / ('%s_P094_df_product_category.csv' % prefix), index=False, header=True, encoding='UTF-8')

CPU times: user 58.1 ms, sys: 0 ns, total: 58.1 ms
Wall time: 83.2 ms


---
> P-095: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはCP932

In [205]:
%%time
_pro.to_csv('./data/P095_df_product_category.csv', index=False, header=True, encoding='CP932')

CPU times: user 58.7 ms, sys: 0 ns, total: 58.7 ms
Wall time: 84.7 ms


## Ver1.1

In [206]:
%%time
_df.to_csv(data_path / ('%s_P095_df_product_category.csv' % prefix), index=False, header=True, encoding='CP932')

CPU times: user 48.9 ms, sys: 9.19 ms, total: 58.1 ms
Wall time: 80.7 ms


---
> P-096: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ無し
> - 文字コードはUTF-8

In [207]:
%%time
_pro.to_csv('./data/P096_df_product_category.csv', index=False, header=False, encoding='UTF-8')

CPU times: user 53.2 ms, sys: 853 µs, total: 54 ms
Wall time: 76.9 ms


## Ver1.1

In [208]:
%%time
_df.to_csv(data_path / ('%s_P096_df_product_category.csv' % prefix), index=False, header=False, encoding='UTF-8')

CPU times: user 51.5 ms, sys: 2.16 ms, total: 53.7 ms
Wall time: 66.5 ms


---
> P-097: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [209]:
%%time
_df = pd.read_csv('./data/P094_df_product_category.csv', header=0, encoding='UTF-8')
_df.head(10)

CPU times: user 14 ms, sys: 10.4 ms, total: 24.3 ms
Wall time: 41 ms


,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_major_name,category_medium_name,category_small_name
0,P040101001,4,401,40101,198.0,149.0,惣菜,御飯類,弁当類
1,P040101002,4,401,40101,218.0,164.0,惣菜,御飯類,弁当類
2,P040101003,4,401,40101,230.0,173.0,惣菜,御飯類,弁当類
3,P040101004,4,401,40101,248.0,186.0,惣菜,御飯類,弁当類
4,P040101005,4,401,40101,268.0,201.0,惣菜,御飯類,弁当類
5,P040101006,4,401,40101,298.0,224.0,惣菜,御飯類,弁当類
6,P040101007,4,401,40101,338.0,254.0,惣菜,御飯類,弁当類
7,P040101008,4,401,40101,420.0,315.0,惣菜,御飯類,弁当類
8,P040101009,4,401,40101,498.0,374.0,惣菜,御飯類,弁当類
9,P040101010,4,401,40101,580.0,435.0,惣菜,御飯類,弁当類


## Ver1.1

In [210]:
%%time
_tmp = pd.read_csv(data_path / ('%s_P094_df_product_category.csv' % prefix), header=0, encoding='UTF-8')
_tmp.head(10)

CPU times: user 22.4 ms, sys: 397 µs, total: 22.8 ms
Wall time: 30 ms


,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_major_name,category_medium_name,category_small_name
0,P040101001,4,401,40101,198.0,149.0,惣菜,御飯類,弁当類
1,P040101002,4,401,40101,218.0,164.0,惣菜,御飯類,弁当類
2,P040101003,4,401,40101,230.0,173.0,惣菜,御飯類,弁当類
3,P040101004,4,401,40101,248.0,186.0,惣菜,御飯類,弁当類
4,P040101005,4,401,40101,268.0,201.0,惣菜,御飯類,弁当類
5,P040101006,4,401,40101,298.0,224.0,惣菜,御飯類,弁当類
6,P040101007,4,401,40101,338.0,254.0,惣菜,御飯類,弁当類
7,P040101008,4,401,40101,420.0,315.0,惣菜,御飯類,弁当類
8,P040101009,4,401,40101,498.0,374.0,惣菜,御飯類,弁当類
9,P040101010,4,401,40101,580.0,435.0,惣菜,御飯類,弁当類


---
> P-098: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ無し
> - 文字コードはUTF-8

In [211]:
%%time
_df = pd.read_csv('./data/P096_df_product_category.csv', header=None, encoding="UTF-8")
_df.head(10)

CPU times: user 22.5 ms, sys: 404 µs, total: 22.9 ms
Wall time: 28.9 ms


,0,1,2,3,4,5,6,7,8
0,P040101001,4,401,40101,198.0,149.0,惣菜,御飯類,弁当類
1,P040101002,4,401,40101,218.0,164.0,惣菜,御飯類,弁当類
2,P040101003,4,401,40101,230.0,173.0,惣菜,御飯類,弁当類
3,P040101004,4,401,40101,248.0,186.0,惣菜,御飯類,弁当類
4,P040101005,4,401,40101,268.0,201.0,惣菜,御飯類,弁当類
5,P040101006,4,401,40101,298.0,224.0,惣菜,御飯類,弁当類
6,P040101007,4,401,40101,338.0,254.0,惣菜,御飯類,弁当類
7,P040101008,4,401,40101,420.0,315.0,惣菜,御飯類,弁当類
8,P040101009,4,401,40101,498.0,374.0,惣菜,御飯類,弁当類
9,P040101010,4,401,40101,580.0,435.0,惣菜,御飯類,弁当類


## Ver1.1

In [212]:
%%time
_tmp = pd.read_csv(data_path / ('%s_P096_df_product_category.csv' % prefix), header=None, encoding='UTF-8')
_tmp.head(10)

CPU times: user 12.4 ms, sys: 9.73 ms, total: 22.2 ms
Wall time: 29.3 ms


,0,1,2,3,4,5,6,7,8
0,P040101001,4,401,40101,198.0,149.0,惣菜,御飯類,弁当類
1,P040101002,4,401,40101,218.0,164.0,惣菜,御飯類,弁当類
2,P040101003,4,401,40101,230.0,173.0,惣菜,御飯類,弁当類
3,P040101004,4,401,40101,248.0,186.0,惣菜,御飯類,弁当類
4,P040101005,4,401,40101,268.0,201.0,惣菜,御飯類,弁当類
5,P040101006,4,401,40101,298.0,224.0,惣菜,御飯類,弁当類
6,P040101007,4,401,40101,338.0,254.0,惣菜,御飯類,弁当類
7,P040101008,4,401,40101,420.0,315.0,惣菜,御飯類,弁当類
8,P040101009,4,401,40101,498.0,374.0,惣菜,御飯類,弁当類
9,P040101010,4,401,40101,580.0,435.0,惣菜,御飯類,弁当類


---
> P-099: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はTSV（タブ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [224]:
_pro.to_csv('./data/P099_df_product_category.tsv', index=False, sep='\t', header=True, encoding='UTF-8')

## Ver1.1

In [198]:
_df.to_csv(data_path / ('%s_P099_df_product_category.tsv' % prefix), index=False, sep='\t', header=True, encoding='UTF-8')

---
> P-100: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はTSV（タブ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [213]:
%time
_df = pd.read_csv('./data/P099_df_product_category.tsv', sep='\t', header=0, encoding='UTF-8')

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 4.77 µs


In [214]:
%time
_df = pd.read_table('./data/P099_df_product_category.tsv', header=0, encoding='UTF-8')

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 4.77 µs


In [215]:
_df.head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_major_name,category_medium_name,category_small_name
0,P040101001,4,401,40101,198.0,149.0,惣菜,御飯類,弁当類
1,P040101002,4,401,40101,218.0,164.0,惣菜,御飯類,弁当類
2,P040101003,4,401,40101,230.0,173.0,惣菜,御飯類,弁当類
3,P040101004,4,401,40101,248.0,186.0,惣菜,御飯類,弁当類
4,P040101005,4,401,40101,268.0,201.0,惣菜,御飯類,弁当類
5,P040101006,4,401,40101,298.0,224.0,惣菜,御飯類,弁当類
6,P040101007,4,401,40101,338.0,254.0,惣菜,御飯類,弁当類
7,P040101008,4,401,40101,420.0,315.0,惣菜,御飯類,弁当類
8,P040101009,4,401,40101,498.0,374.0,惣菜,御飯類,弁当類
9,P040101010,4,401,40101,580.0,435.0,惣菜,御飯類,弁当類


## Ver1.1

In [216]:
%%time
_tmp = pd.read_csv(data_path / ('%s_P099_df_product_category.tsv' % prefix), sep='\t', header=0, encoding='UTF-8')
_tmp.head(10)

CPU times: user 16 ms, sys: 10.1 ms, total: 26.1 ms
Wall time: 31.9 ms


,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_major_name,category_medium_name,category_small_name
0,P040101001,4,401,40101,198.0,149.0,惣菜,御飯類,弁当類
1,P040101002,4,401,40101,218.0,164.0,惣菜,御飯類,弁当類
2,P040101003,4,401,40101,230.0,173.0,惣菜,御飯類,弁当類
3,P040101004,4,401,40101,248.0,186.0,惣菜,御飯類,弁当類
4,P040101005,4,401,40101,268.0,201.0,惣菜,御飯類,弁当類
5,P040101006,4,401,40101,298.0,224.0,惣菜,御飯類,弁当類
6,P040101007,4,401,40101,338.0,254.0,惣菜,御飯類,弁当類
7,P040101008,4,401,40101,420.0,315.0,惣菜,御飯類,弁当類
8,P040101009,4,401,40101,498.0,374.0,惣菜,御飯類,弁当類
9,P040101010,4,401,40101,580.0,435.0,惣菜,御飯類,弁当類


### pickle

In [201]:
%%time
_df.to_pickle(data_path / ('%s_df.pkl' % prefix))

CPU times: user 11.3 ms, sys: 368 µs, total: 11.6 ms
Wall time: 20.6 ms


In [218]:
%%time
_tmp = pd.read_pickle(data_path / ('%s_df.pkl' % prefix))
_tmp.head(10)

CPU times: user 8.62 ms, sys: 4.86 ms, total: 13.5 ms
Wall time: 19.7 ms


,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_major_name,category_medium_name,category_small_name
0,P040101001,04,0401,040101,198.0,149.0,惣菜,御飯類,弁当類
1,P040101002,04,0401,040101,218.0,164.0,惣菜,御飯類,弁当類
2,P040101003,04,0401,040101,230.0,173.0,惣菜,御飯類,弁当類
3,P040101004,04,0401,040101,248.0,186.0,惣菜,御飯類,弁当類
4,P040101005,04,0401,040101,268.0,201.0,惣菜,御飯類,弁当類
5,P040101006,04,0401,040101,298.0,224.0,惣菜,御飯類,弁当類
6,P040101007,04,0401,040101,338.0,254.0,惣菜,御飯類,弁当類
7,P040101008,04,0401,040101,420.0,315.0,惣菜,御飯類,弁当類
8,P040101009,04,0401,040101,498.0,374.0,惣菜,御飯類,弁当類
9,P040101010,04,0401,040101,580.0,435.0,惣菜,御飯類,弁当類


**（松尾コメント）**\
シリアライズ（直列化）を使用し、余計なオプションを指定する必要がない（そのまま保存）\
※処理時間も早め

# これで１００本終わりです。おつかれさまでした！